In [1]:
import numpy as np
import torch
import datetime
from fl_simu import start_experiment
from trees import TreeDataset
import pandas as pd
import numpy as np
import torch
import datetime

D:\FD\venv\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] 找不到指定的程序。'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
df = pd.read_csv('./dataset/data.csv').astype('float32')
df['label'] = df['label'].replace(21, 19)

X = df.drop('label', axis=1)
y = df['label']

X = torch.tensor(X.values, dtype=torch.float32)
y = torch.tensor(y.values, dtype=torch.int)
print(y.shape)

torch.Size([2703])


In [3]:
def do_fl (split, X=X, y=y):
    start = datetime.datetime.now()

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    flag = 1

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X, y)):
        print(f"Fold {fold_idx + 1}:")
        if flag == 2:
            break
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        class_1_indices = torch.where(y_train == 1)[0]
        class_17_indices = torch.where(y_train == 17)[0]
        class_18_indices = torch.where(y_train== 18)[0]
        class_19_indices = torch.where(y_train== 19)[0]
        class_3_indices = torch.where(y_train== 3)[0]
        class_16_indices = torch.where(y_train== 16)[0]
        class_2_indices = torch.where(y_train==2)[0]
        class_0_indices = torch.where(y_train==0)[0]

        class_1_samples = X_train[class_1_indices]
        class_17_samples = X_train[class_17_indices]
        class_18_samples = X_train[class_18_indices]
        class_19_samples = X_train[class_19_indices]
        class_3_samples = X_train[class_3_indices]
        class_16_samples = X_train[class_16_indices]
        class_2_samples = X_train[class_2_indices]
        class_0_samples = X_train[class_0_indices]

        count_1= (50 - class_1_samples.size()[0]) // class_1_samples.size()[0] + 1
        for i in range(count_1):
            X_train = torch.cat((X_train, class_1_samples), dim=0)
            y_train = torch.cat((y_train, torch.full((class_1_samples.size()[0],), 1)), dim=0)

        count_17= (50 - class_17_samples.size()[0]) // class_17_samples.size()[0] + 1
        for i in range(count_17):
            X_train = torch.cat((X_train, class_17_samples), dim=0)
            y_train = torch.cat((y_train, torch.full((class_17_samples.size()[0], ), 17)), dim=0)

        count_18= (50 - class_18_samples.size()[0]) // class_18_samples.size()[0] + 1
        for i in range(count_18):
            X_train = torch.cat((X_train, class_18_samples), dim=0)
            y_train = torch.cat((y_train, torch.full((class_18_samples.size()[0], ), 18)), dim=0)

        count_19= (50 - class_19_samples.size()[0]) // class_19_samples.size()[0] + 1
        for i in range(count_19):
            X_train = torch.cat((X_train, class_19_samples), dim=0)
            y_train = torch.cat((y_train, torch.full((class_19_samples.size()[0], ), 19)), dim=0)

        count_3= (50 - class_3_samples.size()[0]) // class_3_samples.size()[0] + 1
        for i in range(count_3):
            X_train = torch.cat((X_train, class_3_samples), dim=0)
            y_train = torch.cat((y_train, torch.full((class_3_samples.size()[0], ), 3)), dim=0)

        count_16= (50 - class_16_samples.size()[0]) // class_16_samples.size()[0] + 1
        for i in range(count_16):
            X_train = torch.cat((X_train, class_16_samples), dim=0)
            y_train = torch.cat((y_train, torch.full((class_16_samples.size()[0], ), 16)), dim=0)

        count_2= (50 - class_2_samples.size()[0]) // class_2_samples.size()[0] + 1
        for i in range(count_2):
            X_train = torch.cat((X_train, class_2_samples), dim=0)
            y_train = torch.cat((y_train, torch.full((class_2_samples.size()[0], ), 2)), dim=0)

        count_0= (50 - class_0_samples.size()[0]) // class_0_samples.size()[0] + 1
        for i in range(count_0):
            X_train = torch.cat((X_train, class_0_samples), dim=0)
            y_train = torch.cat((y_train, torch.full((class_0_samples.size()[0], ), 0)), dim=0)

        class_4_indices = torch.where(y_train==4)[0]
        class_4_samples = X_train[class_4_indices]

        shuffled_indices = torch.randperm(class_4_samples.shape[0])
        class_4_samples = class_4_samples[shuffled_indices]

        for n in range(5):
            X_train = torch.cat((X_train, class_4_samples[n].unsqueeze(0)), dim=0)

        y_train = torch.cat((y_train, torch.full((5, ), 4)), dim=0)


        trainset = TreeDataset(np.array(X_train, copy=True), np.array(y_train, copy=True))
        testset = TreeDataset(np.array(X_test, copy=True), np.array(y_test, copy=True))

        start = datetime.datetime.now()
        start_experiment(
            split=split,
            trainset=trainset,
            testset=testset,
            num_rounds=15, # 8
            client_tree_num=40,
            client_pool_size=len(split),
            num_iterations=2, #6
            batch_size=2000,
            fraction_fit=0.5, #1
            min_fit_clients=2,
            val_ratio=0.0,
            client_num=len(split),
            task_type = 'multiclass',
            lr=0.001,
            colsample_bytree=1,
            subsample=1,
            max_depth=6,
            min_child_weight=1,
        )

        flag += 1

    end = datetime.datetime.now()
    time = end - start

    print('split', split)
    print('Training time', time)

In [5]:
split = [0.5, 0.5]
do_fl(split)

WARNING flwr 2023-09-04 02:20:23,722 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 02:20:23,724 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


Fold 1:
torch.Size([2700, 22])
torch.Size([2920, 22])
Data partitioned across 2 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 2 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.


2023-09-04 02:20:34,215	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 02:20:38,988 | app.py:179 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 824330649.0, 'memory': 1648661300.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-09-04 02:20:38,989 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 02:20:38,990 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 02:20:38,991 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=3976) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=3976) 
(launch_and_get_parameters pid=3976) Stack (most recent call first):
(launch_and_get_parameters pid=3976)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=3976)   File "D:\FD\venv\lib\site-packages\torch\_ops

(launch_and_get_parameters pid=3976) multiclass client


INFO flwr 2023-09-04 02:20:52,930 | fl_server.py:292 | Received initial parameters from one random client
INFO flwr 2023-09-04 02:20:52,932 | fl_server.py:84 | Evaluating initial parameters


...

Evaluation on the server: test_mse=1.9057, test_auc=0.9000
Configuring round 15
(launch_and_fit pid=3976) multiclass client
(launch_and_fit pid=3976) multiclass client
(launch_and_fit pid=3976) Client 0 aggregated_trees length 2
(launch_and_fit pid=3976) Client 0: recieved 2 trees
(launch_and_fit pid=26832) Client 0: recieved 2 trees
(launch_and_fit pid=3976) Client 0: training for 2 iterations/updates
(launch_and_fit pid=26832) Client 0: training for 2 iterations/updates
(launch_and_fit pid=26832) Client 0: training for 2 iterations/updates


DEBUG flwr 2023-09-04 02:45:30,825 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 02:45:44,667 | fl_server.py:112 | fit progress: (15, 2.1256931608133085, {'auc': 0.9012371838687628}, 1482.8447309000185)
INFO flwr 2023-09-04 02:45:44,669 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 02:45:44,670 | fl_server.py:140 | FL finished in 1482.8479343999934
INFO flwr 2023-09-04 02:45:44,672 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 02:45:44,674 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 02:45:44,675 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 02:45:44,676 | app.py:228 | app_fit: losses_centralized [(0, 124.14417744916821), (1, 2.5268022181146024), (2, 2.2939001848428835), (3, 2.2476894639556377), (4, 2.088724584103512), (5, 2.0462107208872458), (6, 1.8835489833641406), (7, 1.9537892791127542), (8, 2.0628465804066543), (9, 1.922365988909427), (10, 2.086876155268022), (11, 1.9815157116451017), (12, 1.8428835489833642), (13, 1.9038817005

Evaluation on the server: test_mse=2.1257, test_auc=0.9012
History (loss, centralized):
	round 0: 124.14417744916821
	round 1: 2.5268022181146024
	round 2: 2.2939001848428835
	round 3: 2.2476894639556377
	round 4: 2.088724584103512
	round 5: 2.0462107208872458
	round 6: 1.8835489833641406
	round 7: 1.9537892791127542
	round 8: 2.0628465804066543
	round 9: 1.922365988909427
	round 10: 2.086876155268022
	round 11: 1.9815157116451017
	round 12: 1.8428835489833642
	round 13: 1.9038817005545285
	round 14: 1.9057301293900184
	round 15: 2.1256931608133085
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8754135338345864), (2, 0.8841011619958989), (3, 0.8854887218045112), (4, 0.8969514695830485), (5, 0.9022214627477786), (6, 0.9077375256322625), (7, 0.9040396445659604), (8, 0.9077990430622009), (9, 0.9079220779220779), (10, 0.9038550922761448), (11, 0.9018523581681476), (12, 0.9073069036226931), (13, 0.9056732740943266), (14, 0.9000341763499657), (15, 0.9012371838687628)]}
[(0, 124.144

In [6]:
split = [0.6, 0.4]
do_fl(split)

WARNING flwr 2023-09-04 02:45:44,818 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 02:45:44,820 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


Fold 1:
torch.Size([3300, 22])
torch.Size([2320, 22])
Data partitioned across 2 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 2 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.
(launch_and_fit pid=26832) Client 1: training for 2 iterations/updates


2023-09-04 02:45:58,768	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 02:46:04,445 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 986861568.0, 'node:127.0.0.1': 1.0, 'memory': 1973723136.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-09-04 02:46:04,446 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 02:46:04,447 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 02:46:04,448 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=24812) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=24812) 
(launch_and_get_parameters pid=24812) Stack (most recent call first):
(launch_and_get_parameters pid=24812)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=24812)   File "D:\FD\venv\lib\site-packages\torch

(launch_and_get_parameters pid=24812) multiclass client


INFO flwr 2023-09-04 02:46:20,732 | fl_server.py:292 | Received initial parameters from one random client
INFO flwr 2023-09-04 02:46:20,734 | fl_server.py:84 | Evaluating initial parameters


...

Evaluation on the server: test_mse=1.7893, test_auc=0.9167
Configuring round 15
(launch_and_fit pid=24812) multiclass client
(launch_and_fit pid=24868) multiclass client
(launch_and_fit pid=24812) Client 1 aggregated_trees length 2
(launch_and_fit pid=24812) Client 1: recieved 2 trees
(launch_and_fit pid=24868) Client 0: recieved 2 trees
(launch_and_fit pid=24812) Client 1: training for 2 iterations/updates
(launch_and_fit pid=24868) Client 1: training for 2 iterations/updates
(launch_and_fit pid=24868) Client 1: training for 2 iterations/updates


DEBUG flwr 2023-09-04 03:00:07,365 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 03:00:18,929 | fl_server.py:112 | fit progress: (15, 1.8151571164510167, {'auc': 0.9147641831852358}, 825.5611779000028)
INFO flwr 2023-09-04 03:00:18,931 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 03:00:18,934 | fl_server.py:140 | FL finished in 825.5658147000067
INFO flwr 2023-09-04 03:00:18,936 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 03:00:18,937 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 03:00:18,938 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 03:00:18,939 | app.py:228 | app_fit: losses_centralized [(0, 85.452865064695), (1, 2.598890942698706), (2, 2.5804066543438076), (3, 2.358595194085028), (4, 2.142329020332717), (5, 2.1182994454713495), (6, 2.0702402957486137), (7, 2.075785582255083), (8, 2.053604436229205), (9, 2.022181146025878), (10, 2.0351201478743066), (11, 1.8909426987060998), (12, 1.8964879852125693), (13, 2.1072088724584104),

Evaluation on the server: test_mse=1.8152, test_auc=0.9148
History (loss, centralized):
	round 0: 85.452865064695
	round 1: 2.598890942698706
	round 2: 2.5804066543438076
	round 3: 2.358595194085028
	round 4: 2.142329020332717
	round 5: 2.1182994454713495
	round 6: 2.0702402957486137
	round 7: 2.075785582255083
	round 8: 2.053604436229205
	round 9: 2.022181146025878
	round 10: 2.0351201478743066
	round 11: 1.8909426987060998
	round 12: 1.8964879852125693
	round 13: 2.1072088724584104
	round 14: 1.789279112754159
	round 15: 1.8151571164510167
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8758441558441559), (2, 0.8874914559125084), (3, 0.8772317156527683), (4, 0.8922693096377307), (5, 0.9002187286397813), (6, 0.9072453861927546), (7, 0.9115584415584416), (8, 0.9057347915242652), (9, 0.9109432672590567), (10, 0.9147641831852358), (11, 0.9190772385509227), (12, 0.9133766233766234), (13, 0.9012987012987014), (14, 0.9167053998632946), (15, 0.9147641831852358)]}
[(0, 85.45286506469

In [7]:
split = [0.8, 0.2]
do_fl(split)

WARNING flwr 2023-09-04 03:00:19,044 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 03:00:19,045 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


Fold 1:
torch.Size([4500, 22])
torch.Size([1120, 22])
Data partitioned across 2 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 2 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.
(launch_and_fit pid=24868) Client 0: training for 2 iterations/updates


2023-09-04 03:00:27,546	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 03:00:32,150 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 1110535372.0, 'node:127.0.0.1': 1.0, 'memory': 2221070747.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-09-04 03:00:32,151 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 03:00:32,152 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 03:00:32,153 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=26832) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=26832) 
(launch_and_get_parameters pid=26832) Stack (most recent call first):
(launch_and_get_parameters pid=26832)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=26832)   File "D:\FD\venv\lib\site-packages\torc

(launch_and_get_parameters pid=26832) multiclass client


INFO flwr 2023-09-04 03:00:43,576 | fl_server.py:292 | Received initial parameters from one random client
INFO flwr 2023-09-04 03:00:43,577 | fl_server.py:84 | Evaluating initial parameters


...

Evaluation on the server: test_mse=1.9686, test_auc=0.9150
Configuring round 15
(launch_and_fit pid=26832) multiclass client
(launch_and_fit pid=26832) Client 1 aggregated_trees length 2
(launch_and_fit pid=26832) Client 1: recieved 2 trees
(launch_and_fit pid=19644) Client 0: recieved 2 trees
(launch_and_fit pid=19644) Client 0: recieved 2 trees
(launch_and_fit pid=19644) Client 0: recieved 2 trees
(launch_and_fit pid=26832) Client 1: training for 2 iterations/updates
(launch_and_fit pid=19644) Client 0: training for 2 iterations/updates


DEBUG flwr 2023-09-04 03:16:18,474 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 03:16:34,699 | fl_server.py:112 | fit progress: (15, 1.7929759704251387, {'auc': 0.9134996582365003}, 938.6219850999769)
INFO flwr 2023-09-04 03:16:34,700 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 03:16:34,701 | fl_server.py:140 | FL finished in 938.6240743999952
INFO flwr 2023-09-04 03:16:34,702 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 03:16:34,703 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 03:16:34,704 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 03:16:34,705 | app.py:228 | app_fit: losses_centralized [(0, 101.01109057301294), (1, 2.809611829944547), (2, 2.3160813308687613), (3, 2.162661737523105), (4, 1.9981515711645101), (5, 2.1386321626617377), (6, 2.1312384473197783), (7, 2.22181146025878), (8, 2.1792975970425137), (9, 2.1829944547134934), (10, 2.014787430683919), (11, 2.0055452865064693), (12, 1.990757855822551), (13, 2.11460258780037)

Evaluation on the server: test_mse=1.7930, test_auc=0.9135
History (loss, centralized):
	round 0: 101.01109057301294
	round 1: 2.809611829944547
	round 2: 2.3160813308687613
	round 3: 2.162661737523105
	round 4: 1.9981515711645101
	round 5: 2.1386321626617377
	round 6: 2.1312384473197783
	round 7: 2.22181146025878
	round 8: 2.1792975970425137
	round 9: 2.1829944547134934
	round 10: 2.014787430683919
	round 11: 2.0055452865064693
	round 12: 1.990757855822551
	round 13: 2.11460258780037
	round 14: 1.9685767097966729
	round 15: 1.7929759704251387
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8203485987696514), (2, 0.8891866028708135), (3, 0.8893711551606288), (4, 0.8934381408065618), (5, 0.8986466165413534), (6, 0.902282980177717), (7, 0.9059808612440191), (8, 0.9008339029391661), (9, 0.9096787423103212), (10, 0.9113738892686261), (11, 0.9133766233766234), (12, 0.913315105946685), (13, 0.9041626794258373), (14, 0.9150102529049897), (15, 0.9134996582365003)]}
[(0, 101.0110905730

In [8]:
split = [0.34, 0.33, 0.33]
do_fl(split)

WARNING flwr 2023-09-04 03:16:34,852 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 03:16:34,854 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


Fold 1:
torch.Size([1740, 22])
torch.Size([1920, 22])
torch.Size([1860, 22])
Data partitioned across 3 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 3 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.


2023-09-04 03:16:44,007	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 03:16:51,396 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 1067472076.0, 'node:127.0.0.1': 1.0, 'memory': 2134944155.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-09-04 03:16:51,399 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 03:16:51,401 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 03:16:51,403 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=26800) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=26800) 
(launch_and_get_parameters pid=26800) Stack (most recent call first):
(launch_and_get_parameters pid=26800)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=26800)   File "D:\FD\venv\lib\site-packages\torc

(launch_and_get_parameters pid=26800) multiclass client


INFO flwr 2023-09-04 03:17:00,626 | fl_server.py:292 | Received initial parameters from one random client
INFO flwr 2023-09-04 03:17:00,628 | fl_server.py:84 | Evaluating initial parameters


...

Evaluation on the server: test_mse=2.6580, test_auc=0.8892
Configuring round 15
(launch_and_fit pid=26800) multiclass client
(launch_and_fit pid=28020) multiclass client
(launch_and_fit pid=26800) Client 0 aggregated_trees length 2
(launch_and_fit pid=26800) Client 0: recieved 2 trees
(launch_and_fit pid=26800) Client 0: training for 2 iterations/updates
(launch_and_fit pid=28020) Client 0: training for 2 iterations/updates
(launch_and_fit pid=28020) Client 0: training for 2 iterations/updates
(launch_and_fit pid=28020) Client 0: training for 2 iterations/updates


DEBUG flwr 2023-09-04 03:39:59,001 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 03:40:20,283 | fl_server.py:112 | fit progress: (15, 2.3179297597042514, {'auc': 0.8798496240601503}, 1375.5193902000028)
INFO flwr 2023-09-04 03:40:20,285 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 03:40:20,286 | fl_server.py:140 | FL finished in 1375.5229222000053
INFO flwr 2023-09-04 03:40:20,288 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 03:40:20,290 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 03:40:20,291 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 03:40:20,292 | app.py:228 | app_fit: losses_centralized [(0, 124.14417744916821), (1, 3.521256931608133), (2, 3.0258780036968576), (3, 3.024029574861368), (4, 2.720887245841035), (5, 2.4621072088724585), (6, 2.8262476894639557), (7, 2.532347504621072), (8, 2.499075785582255), (9, 2.469500924214418), (10, 2.66543438077634), (11, 2.4491682070240297), (12, 2.8595194085027726), (13, 2.524953789279112

Evaluation on the server: test_mse=2.3179, test_auc=0.8798
History (loss, centralized):
	round 0: 124.14417744916821
	round 1: 3.521256931608133
	round 2: 3.0258780036968576
	round 3: 3.024029574861368
	round 4: 2.720887245841035
	round 5: 2.4621072088724585
	round 6: 2.8262476894639557
	round 7: 2.532347504621072
	round 8: 2.499075785582255
	round 9: 2.469500924214418
	round 10: 2.66543438077634
	round 11: 2.4491682070240297
	round 12: 2.8595194085027726
	round 13: 2.5249537892791127
	round 14: 2.6580406654343807
	round 15: 2.3179297597042514
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8611756664388243), (2, 0.864688995215311), (3, 0.8734381408065618), (4, 0.8754135338345864), (5, 0.8947641831852358), (6, 0.8836705399863294), (7, 0.8820369104579631), (8, 0.8717156527682844), (9, 0.8880451127819547), (10, 0.8678947368421052), (11, 0.8801572112098428), (12, 0.8815447710184552), (13, 0.8891866028708135), (14, 0.8892481203007518), (15, 0.8798496240601503)]}
[(0, 124.144177449

In [9]:
split = [0.6, 0.2, 0.2]
do_fl(split)

WARNING flwr 2023-09-04 03:40:20,422 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 03:40:20,424 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


Fold 1:
torch.Size([3300, 22])
torch.Size([1200, 22])
torch.Size([1120, 22])
Data partitioned across 3 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 3 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.
(launch_and_fit pid=28020) Client 2: training for 2 iterations/updates


2023-09-04 03:40:28,681	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 03:40:36,222 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 871317504.0, 'node:127.0.0.1': 1.0, 'memory': 1742635008.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-09-04 03:40:36,225 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 03:40:36,226 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 03:40:36,228 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=10268) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=10268) 
(launch_and_get_parameters pid=10268) Stack (most recent call first):
(launch_and_get_parameters pid=10268)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=10268)   File "D:\FD\venv\lib\site-packages\torch

(launch_and_get_parameters pid=10268) multiclass client


INFO flwr 2023-09-04 03:40:43,721 | fl_server.py:292 | Received initial parameters from one random client
INFO flwr 2023-09-04 03:40:43,722 | fl_server.py:84 | Evaluating initial parameters


...

Evaluation on the server: test_mse=2.7431, test_auc=0.8876
Configuring round 15
(launch_and_fit pid=24964) multiclass client
(launch_and_fit pid=24964) multiclass client
(launch_and_fit pid=10268) Client 2 aggregated_trees length 2
(launch_and_fit pid=10268) Client 2: recieved 2 trees
(launch_and_fit pid=10268) Client 2: training for 2 iterations/updates
(launch_and_fit pid=10268) Client 2: training for 2 iterations/updates
(launch_and_fit pid=24964) Client 2: training for 2 iterations/updates
(launch_and_fit pid=24964) Client 2: training for 2 iterations/updates


DEBUG flwr 2023-09-04 04:01:28,474 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 04:01:46,595 | fl_server.py:112 | fit progress: (15, 2.3937153419593344, {'auc': 0.8706971975393027}, 1245.2382839999918)
INFO flwr 2023-09-04 04:01:46,597 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 04:01:46,598 | fl_server.py:140 | FL finished in 1245.2412417999876
INFO flwr 2023-09-04 04:01:46,599 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 04:01:46,600 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 04:01:46,601 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 04:01:46,603 | app.py:228 | app_fit: losses_centralized [(0, 124.14417744916821), (1, 3.7301293900184844), (2, 2.7060998151571165), (3, 2.266173752310536), (4, 2.979667282809612), (5, 4.11275415896488), (6, 1.9870609981515712), (7, 4.373382624768946), (8, 2.088724584103512), (9, 1.8724584103512014), (10, 3.789279112754159), (11, 3.168207024029575), (12, 1.9944547134935304), (13, 1.826247689463955

Evaluation on the server: test_mse=2.3937, test_auc=0.8707
History (loss, centralized):
	round 0: 124.14417744916821
	round 1: 3.7301293900184844
	round 2: 2.7060998151571165
	round 3: 2.266173752310536
	round 4: 2.979667282809612
	round 5: 4.11275415896488
	round 6: 1.9870609981515712
	round 7: 4.373382624768946
	round 8: 2.088724584103512
	round 9: 1.8724584103512014
	round 10: 3.789279112754159
	round 11: 3.168207024029575
	round 12: 1.9944547134935304
	round 13: 1.8262476894639557
	round 14: 2.743068391866913
	round 15: 2.3937153419593344
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8673684210526316), (2, 0.8798496240601503), (3, 0.8913123718386876), (4, 0.8857347915242653), (5, 0.836678058783322), (6, 0.9022214627477786), (7, 0.8409295967190704), (8, 0.9046548188653452), (9, 0.9069993164730006), (10, 0.8392344497607656), (11, 0.8576623376623377), (12, 0.9045933014354067), (13, 0.9193233082706767), (14, 0.8876144907723855), (15, 0.8706971975393027)]}
[(0, 124.1441774491

In [10]:
split = [0.8, 0.1, 0.1]
do_fl(split)

WARNING flwr 2023-09-04 04:01:46,734 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 04:01:46,736 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


Fold 1:
torch.Size([4500, 22])
torch.Size([600, 22])
torch.Size([520, 22])
Data partitioned across 3 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 3 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.
(launch_and_fit pid=24964) Client 1: training for 2 iterations/updates


2023-09-04 04:01:55,157	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 04:02:02,758 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 847740518.0, 'node:127.0.0.1': 1.0, 'memory': 1695481038.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-09-04 04:02:02,761 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 04:02:02,763 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 04:02:02,765 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=21068) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=21068) 
(launch_and_get_parameters pid=21068) Stack (most recent call first):
(launch_and_get_parameters pid=21068)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=21068)   File "D:\FD\venv\lib\site-packages\torch

(launch_and_get_parameters pid=21068) multiclass client


INFO flwr 2023-09-04 04:02:09,102 | fl_server.py:292 | Received initial parameters from one random client
INFO flwr 2023-09-04 04:02:09,103 | fl_server.py:84 | Evaluating initial parameters


...

Evaluation on the server: test_mse=2.1368, test_auc=0.8820
Configuring round 15
(launch_and_fit pid=21068) multiclass client
(launch_and_fit pid=21068) Client 2 aggregated_trees length 2
(launch_and_fit pid=21068) Client 2: recieved 2 trees
(launch_and_fit pid=8268) Client 0: recieved 2 trees
(launch_and_fit pid=8268) Client 0: recieved 2 trees
(launch_and_fit pid=8268) Client 0: recieved 2 trees
(launch_and_fit pid=21068) Client 2: training for 2 iterations/updates
(launch_and_fit pid=8268) Client 0: training for 2 iterations/updates


DEBUG flwr 2023-09-04 04:20:10,124 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 04:20:24,194 | fl_server.py:112 | fit progress: (15, 4.591497227356747, {'auc': 0.8159671907040327}, 1081.4573450000025)
INFO flwr 2023-09-04 04:20:24,196 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 04:20:24,197 | fl_server.py:140 | FL finished in 1081.4594924999983
INFO flwr 2023-09-04 04:20:24,197 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 04:20:24,198 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 04:20:24,199 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 04:20:24,201 | app.py:228 | app_fit: losses_centralized [(0, 122.33086876155268), (1, 5.182994454713493), (2, 4.685767097966728), (3, 4.656192236598891), (4, 4.539741219963031), (5, 2.2365988909426986), (6, 4.360443622920518), (7, 5.079482439926063), (8, 4.310536044362292), (9, 2.5064695009242146), (10, 4.118299445471349), (11, 2.2643253234750462), (12, 2.123844731977819), (13, 4.020332717190388),

Evaluation on the server: test_mse=4.5915, test_auc=0.8160
History (loss, centralized):
	round 0: 122.33086876155268
	round 1: 5.182994454713493
	round 2: 4.685767097966728
	round 3: 4.656192236598891
	round 4: 4.539741219963031
	round 5: 2.2365988909426986
	round 6: 4.360443622920518
	round 7: 5.079482439926063
	round 8: 4.310536044362292
	round 9: 2.5064695009242146
	round 10: 4.118299445471349
	round 11: 2.2643253234750462
	round 12: 2.123844731977819
	round 13: 4.020332717190388
	round 14: 2.1367837338262476
	round 15: 4.591497227356747
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.7721531100478468), (2, 0.8214217361585783), (3, 0.8244429254955571), (4, 0.8221326042378674), (5, 0.8915584415584416), (6, 0.822071086807929), (7, 0.8294668489405331), (8, 0.83904989747095), (9, 0.8779084073820916), (10, 0.8468147641831851), (11, 0.8745181134654819), (12, 0.8968284347231716), (13, 0.8371701982228298), (14, 0.8820369104579631), (15, 0.8159671907040327)]}
[(0, 122.33086876155268

In [11]:
split = [0.2, 0.2, 0.2, 0.2, 0.2]
do_fl(split)

WARNING flwr 2023-09-04 04:20:24,349 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 04:20:24,351 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


Fold 1:
torch.Size([1000, 22])
torch.Size([1100, 22])
torch.Size([1200, 22])
torch.Size([1200, 22])
torch.Size([1120, 22])
Data partitioned across 5 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 5 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.


2023-09-04 04:20:32,446	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 04:20:40,712 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 765349478.0, 'node:127.0.0.1': 1.0, 'memory': 1530698958.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-09-04 04:20:40,714 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 04:20:40,717 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 04:20:40,719 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=7712) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=7712) 
(launch_and_get_parameters pid=7712) Stack (most recent call first):
(launch_and_get_parameters pid=7712)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=7712)   File "D:\FD\venv\lib\site-packages\torch\_ops

(launch_and_get_parameters pid=7712) multiclass client


INFO flwr 2023-09-04 04:20:48,440 | fl_server.py:292 | Received initial parameters from one random client
INFO flwr 2023-09-04 04:20:48,441 | fl_server.py:84 | Evaluating initial parameters


...

Evaluation on the server: test_mse=2.9076, test_auc=0.8633
Configuring round 15
(launch_and_fit pid=7712) multiclass client
(launch_and_fit pid=588) multiclass client
(launch_and_fit pid=7712) Client 1 aggregated_trees length 2
(launch_and_fit pid=7712) Client 1: recieved 2 trees
(launch_and_fit pid=7712) Client 1: training for 2 iterations/updates
(launch_and_fit pid=588) Client 1: training for 2 iterations/updates
(launch_and_fit pid=588) Client 1: training for 2 iterations/updates
(launch_and_fit pid=588) Client 1: training for 2 iterations/updates


DEBUG flwr 2023-09-04 04:46:43,362 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 04:47:17,197 | fl_server.py:112 | fit progress: (15, 2.6062846580406656, {'auc': 0.874395078605605}, 1557.3468561999907)
INFO flwr 2023-09-04 04:47:17,199 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 04:47:17,200 | fl_server.py:140 | FL finished in 1557.3498013999779
INFO flwr 2023-09-04 04:47:17,201 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 04:47:17,203 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 04:47:17,204 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 04:47:17,206 | app.py:228 | app_fit: losses_centralized [(0, 111.41404805914972), (1, 4.00554528650647), (2, 2.985212569316081), (3, 3.229205175600739), (4, 2.9408502772643255), (5, 3.9611829944547137), (6, 3.2846580406654344), (7, 3.0351201478743066), (8, 2.6173752310536043), (9, 3.5582255083179297), (10, 2.7966728280961184), (11, 3.1164510166358594), (12, 3.4491682070240297), (13, 3.232902033271

Evaluation on the server: test_mse=2.6063, test_auc=0.8744
History (loss, centralized):
	round 0: 111.41404805914972
	round 1: 4.00554528650647
	round 2: 2.985212569316081
	round 3: 3.229205175600739
	round 4: 2.9408502772643255
	round 5: 3.9611829944547137
	round 6: 3.2846580406654344
	round 7: 3.0351201478743066
	round 8: 2.6173752310536043
	round 9: 3.5582255083179297
	round 10: 2.7966728280961184
	round 11: 3.1164510166358594
	round 12: 3.4491682070240297
	round 13: 3.232902033271719
	round 14: 2.9075785582255085
	round 15: 2.6062846580406656
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8575393028024607), (2, 0.8633014354066986), (3, 0.860745044429255), (4, 0.8689405331510595), (5, 0.8727614490772385), (6, 0.8804032809295967), (7, 0.8751948051948052), (8, 0.8645659603554341), (9, 0.8593574846206425), (10, 0.8751948051948052), (11, 0.862036910457963), (12, 0.8674914559125085), (13, 0.8825290498974708), (14, 0.8633014354066986), (15, 0.874395078605605)]}
[(0, 111.41404805

In [12]:
split = [0.6, 0.1, 0.1, 0.1, 0.1]
do_fl(split)

WARNING flwr 2023-09-04 04:47:17,373 | app.py:210 | Both server and strategy were provided, ignoring strategy


Fold 1:
torch.Size([3300, 22])
torch.Size([600, 22])
torch.Size([600, 22])
torch.Size([600, 22])
torch.Size([520, 22])
Data partitioned across 5 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 5 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.


INFO flwr 2023-09-04 04:47:17,376 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


(launch_and_fit pid=588) Client 3: training for 2 iterations/updates


2023-09-04 04:47:25,992	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 04:47:33,939 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 729236275.0, 'node:127.0.0.1': 1.0, 'memory': 1458472551.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-09-04 04:47:33,941 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 04:47:33,943 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 04:47:33,945 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=27636) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=27636) 
(launch_and_get_parameters pid=27636) Stack (most recent call first):
(launch_and_get_parameters pid=27636)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=27636)   File "D:\FD\venv\lib\site-packages\torch

...

Evaluation on the server: test_mse=3.7301, test_auc=0.8584
Configuring round 15
(launch_and_fit pid=27636) multiclass client
(launch_and_fit pid=25536) multiclass client
(launch_and_fit pid=25536) Client 4 aggregated_trees length 2
(launch_and_fit pid=25536) Client 4: recieved 2 trees
(launch_and_fit pid=25536) Client 4: training for 2 iterations/updates
(launch_and_fit pid=25536) Client 4: training for 2 iterations/updates
(launch_and_fit pid=27636) Client 4: training for 2 iterations/updates
(launch_and_fit pid=27636) Client 4: training for 2 iterations/updates


DEBUG flwr 2023-09-04 05:08:54,767 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 05:09:23,191 | fl_server.py:112 | fit progress: (15, 3.500924214417745, {'auc': 0.8518386876281614}, 1247.7465533000068)
INFO flwr 2023-09-04 05:09:23,194 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 05:09:23,197 | fl_server.py:140 | FL finished in 1247.7525508000108
INFO flwr 2023-09-04 05:09:23,199 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 05:09:23,201 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 05:09:23,203 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 05:09:23,205 | app.py:228 | app_fit: losses_centralized [(0, 87.84842883548983), (1, 2.852125693160813), (2, 5.513863216266174), (3, 4.606284658040665), (4, 3.611829944547135), (5, 3.497227356746765), (6, 3.6931608133086877), (7, 3.4750462107208873), (8, 3.8262476894639557), (9, 3.622920517560074), (10, 3.6432532347504623), (11, 6.051756007393715), (12, 3.567467652495379), (13, 3.613678373382625),

Evaluation on the server: test_mse=3.5009, test_auc=0.8518
History (loss, centralized):
	round 0: 87.84842883548983
	round 1: 2.852125693160813
	round 2: 5.513863216266174
	round 3: 4.606284658040665
	round 4: 3.611829944547135
	round 5: 3.497227356746765
	round 6: 3.6931608133086877
	round 7: 3.4750462107208873
	round 8: 3.8262476894639557
	round 9: 3.622920517560074
	round 10: 3.6432532347504623
	round 11: 6.051756007393715
	round 12: 3.567467652495379
	round 13: 3.613678373382625
	round 14: 3.7301293900184844
	round 15: 3.500924214417745
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8796650717703349), (2, 0.8256459330143541), (3, 0.820622009569378), (4, 0.8483253588516746), (5, 0.8419138755980861), (6, 0.8385577580314423), (7, 0.8544565960355435), (8, 0.8540259740259742), (9, 0.8382501708817498), (10, 0.8423444976076555), (11, 0.8294053315105947), (12, 0.8543335611756664), (13, 0.8459808612440192), (14, 0.8584005468215994), (15, 0.8518386876281614)]}
[(0, 87.8484288354898

In [13]:
split = [0.8, 0.05, 0.05, 0.05, 0.05]
do_fl(split) # 0.0001, 0.3, 1 sampling rate

Fold 1:
torch.Size([4500, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([200, 22])
Data partitioned across 5 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 5 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.


WARNING flwr 2023-09-04 05:09:23,380 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 05:09:23,382 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


(launch_and_fit pid=27636) Client 2: training for 2 iterations/updates


2023-09-04 05:09:34,256	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 05:09:42,722 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 651411456.0, 'node:127.0.0.1': 1.0, 'memory': 1302822912.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-09-04 05:09:42,724 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 05:09:42,726 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 05:09:42,727 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=21784) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=21784) 
(launch_and_get_parameters pid=21784) Stack (most recent call first):
(launch_and_get_parameters pid=21784)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=21784)   File "D:\FD\venv\lib\site-packages\torch

(launch_and_get_parameters pid=21784) multiclass client


...

Evaluation on the server: test_mse=5.2107, test_auc=0.8290
Configuring round 15
(launch_and_fit pid=21784) multiclass client
(launch_and_fit pid=15724) multiclass client
(launch_and_fit pid=21784) Client 1 aggregated_trees length 2
(launch_and_fit pid=21784) Client 1: recieved 2 trees
(launch_and_fit pid=21784) Client 1: training for 2 iterations/updates
(launch_and_fit pid=15724) Client 1: training for 2 iterations/updates
(launch_and_fit pid=15724) Client 1: training for 2 iterations/updates
(launch_and_fit pid=15724) Client 1: training for 2 iterations/updates


DEBUG flwr 2023-09-04 05:34:53,165 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 05:35:12,871 | fl_server.py:112 | fit progress: (15, 4.844731977818854, {'auc': 0.8228092959671907}, 1505.0612199999741)
INFO flwr 2023-09-04 05:35:12,873 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 05:35:12,874 | fl_server.py:140 | FL finished in 1505.0645704999915
INFO flwr 2023-09-04 05:35:12,876 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 05:35:12,877 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 05:35:12,879 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 05:35:12,880 | app.py:228 | app_fit: losses_centralized [(0, 104.25323475046211), (1, 5.3012939001848425), (2, 5.066543438077634), (3, 6.859519408502773), (4, 2.4565619223659887), (5, 5.839186691312385), (6, 6.044362292051756), (7, 5.293900184842884), (8, 3.870609981515712), (9, 2.210720887245841), (10, 2.1460258780036967), (11, 6.643253234750462), (12, 6.720887245841035), (13, 2.2421441774491684)

Evaluation on the server: test_mse=4.8447, test_auc=0.8228
History (loss, centralized):
	round 0: 104.25323475046211
	round 1: 5.3012939001848425
	round 2: 5.066543438077634
	round 3: 6.859519408502773
	round 4: 2.4565619223659887
	round 5: 5.839186691312385
	round 6: 6.044362292051756
	round 7: 5.293900184842884
	round 8: 3.870609981515712
	round 9: 2.210720887245841
	round 10: 2.1460258780036967
	round 11: 6.643253234750462
	round 12: 6.720887245841035
	round 13: 2.2421441774491684
	round 14: 5.210720887245841
	round 15: 4.844731977818854
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8051537935748463), (2, 0.7947641831852358), (3, 0.7666985645933013), (4, 0.8673684210526316), (5, 0.8289747095010254), (6, 0.7907928913192072), (7, 0.8312235133287765), (8, 0.8701708817498292), (9, 0.8900478468899522), (10, 0.9079220779220779), (11, 0.7829665071770336), (12, 0.8000683526999317), (13, 0.8972590567327411), (14, 0.8290362269309639), (15, 0.8228092959671907)]}
[(0, 104.25323475046

In [14]:
split = [0.1, 0.1, 0.1, 0.1, 0.1,0.1, 0.1, 0.1, 0.1, 0.1]
do_fl(split)

WARNING flwr 2023-09-04 05:35:13,069 | app.py:210 | Both server and strategy were provided, ignoring strategy


Fold 1:
torch.Size([500, 22])
torch.Size([500, 22])
torch.Size([500, 22])
torch.Size([600, 22])
torch.Size([600, 22])
torch.Size([600, 22])
torch.Size([600, 22])
torch.Size([600, 22])
torch.Size([600, 22])
torch.Size([520, 22])
Data partitioned across 10 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 10 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.


INFO flwr 2023-09-04 05:35:13,072 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


(launch_and_fit pid=15724) Client 2: training for 2 iterations/updates


2023-09-04 05:35:23,875	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 05:35:31,899 | app.py:179 | Flower VCE: Ray initialized with resources: {'object_store_memory': 669265920.0, 'node:127.0.0.1': 1.0, 'memory': 1338531840.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-09-04 05:35:31,902 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 05:35:31,904 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 05:35:31,906 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=27380) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=27380) 
(launch_and_get_parameters pid=27380) Stack (most recent call first):
(launch_and_get_parameters pid=27380)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=27380)   File "D:\FD\venv\lib\site-packages\torch

...

Evaluation on the server: test_mse=4.6987, test_auc=0.8306
Configuring round 15
(launch_and_fit pid=18136) multiclass client
(launch_and_fit pid=18136) Client 3: training for 2 iterations/updates [repeated 3x across cluster]
(launch_and_fit pid=24552) multiclass client
(launch_and_fit pid=24552) Client 9 aggregated_trees length 5
(launch_and_fit pid=24552) Client 9: recieved 5 trees
(launch_and_fit pid=27244) Client 8: training for 2 iterations/updates
(launch_and_fit pid=27244) multiclass client [repeated 3x across cluster]
(launch_and_fit pid=27244) Client 8 aggregated_trees length 5 [repeated 4x across cluster]
(launch_and_fit pid=27244) Client 8: recieved 5 trees [repeated 4x across cluster]
(launch_and_fit pid=24552) Client 9: training for 2 iterations/updates


DEBUG flwr 2023-09-04 06:26:46,611 | fl_server.py:241 | fit_round 15 received 5 results and 0 failures


Server side aggregated 5 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 06:27:35,299 | fl_server.py:112 | fit progress: (15, 4.317929759704251, {'auc': 0.8461995898838004}, 3063.247941800015)
INFO flwr 2023-09-04 06:27:35,301 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 06:27:35,302 | fl_server.py:140 | FL finished in 3063.251369400008
INFO flwr 2023-09-04 06:27:35,304 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 06:27:35,305 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 06:27:35,306 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 06:27:35,307 | app.py:228 | app_fit: losses_centralized [(0, 105.36598890942699), (1, 3.741219963031423), (2, 4.144177449168207), (3, 4.576709796672828), (4, 4.513863216266174), (5, 3.844731977818854), (6, 4.260628465804067), (7, 4.085027726432532), (8, 3.922365988909427), (9, 3.852125693160813), (10, 3.2846580406654344), (11, 4.436229205175601), (12, 4.3345656192236595), (13, 4.401109057301294), (1

Evaluation on the server: test_mse=4.3179, test_auc=0.8462
History (loss, centralized):
	round 0: 105.36598890942699
	round 1: 3.741219963031423
	round 2: 4.144177449168207
	round 3: 4.576709796672828
	round 4: 4.513863216266174
	round 5: 3.844731977818854
	round 6: 4.260628465804067
	round 7: 4.085027726432532
	round 8: 3.922365988909427
	round 9: 3.852125693160813
	round 10: 3.2846580406654344
	round 11: 4.436229205175601
	round 12: 4.3345656192236595
	round 13: 4.401109057301294
	round 14: 4.6987060998151575
	round 15: 4.317929759704251
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8154477101845523), (2, 0.8184962406015037), (3, 0.8426247436773753), (4, 0.8449965823650034), (5, 0.8269104579630895), (6, 0.8485714285714285), (7, 0.8543950786056049), (8, 0.849958988380041), (9, 0.8569514695830486), (10, 0.8592959671907039), (11, 0.8460765550239234), (12, 0.8309159261790842), (13, 0.8469993164730006), (14, 0.8306356801093643), (15, 0.8461995898838004)]}
[(0, 105.3659889094269

In [15]:
split = [0.6, 0.05, 0.05, 0.05, 0.05, 0.04, 0.04, 0.04, 0.04, 0.04]
do_fl(split)

Fold 1:
torch.Size([3300, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([200, 22])
Data partitioned across 10 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 10 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.


WARNING flwr 2023-09-04 06:27:35,484 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 06:27:35,486 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


(launch_and_fit pid=27380) Client 6: training for 2 iterations/updates [repeated 3x across cluster]


(launch_and_fit pid=24552) Windows fatal exception: code 0xc0000139 [repeated 6x across cluster]
(launch_and_fit pid=24552)  [repeated 6x across cluster]
(launch_and_fit pid=24552)   File "<frozen importlib._bootstrap>", line 241 in _call_with_frames_removed [repeated 90x across cluster]
(launch_and_fit pid=24552)   File "<frozen importlib._bootstrap_external>", line 883 in exec_module [repeated 70x across cluster]
(launch_and_fit pid=24552)   File "<frozen importlib._bootstrap>", line 688 in _load_unlocked [repeated 70x across cluster]
(launch_and_fit pid=24552)   File "<frozen importlib._bootstrap>", line 1006 in _find_and_load_unlocked [repeated 80x across cluster]
(launch_and_fit pid=24552)   File "<frozen importlib._bootstrap>", line 1027 in _find_and_load [repeated 79x across cluster]
(launch_and_fit pid=24552)   File "<frozen importlib._bootstrap>", line 1078 in _handle_fromlist [repeated 10x across cluster]
(launch_and_fit pid=24552) Stack (most recent call first): [repeated 3x

(launch_and_get_parameters pid=22256) multiclass client


...

Evaluation on the server: test_mse=6.6451, test_auc=0.8252
Configuring round 15
(launch_and_fit pid=22256) multiclass client
(launch_and_fit pid=26772) Client 9 aggregated_trees length 5
(launch_and_fit pid=26772) Client 9: recieved 5 trees
(launch_and_fit pid=9752) multiclass client [repeated 4x across cluster]
(launch_and_fit pid=1740) Client 0 aggregated_trees length 5 [repeated 4x across cluster]
(launch_and_fit pid=1740) Client 0: recieved 5 trees [repeated 4x across cluster]
(launch_and_fit pid=22256) Client 7: training for 2 iterations/updates
(launch_and_fit pid=1740) Client 0: training for 2 iterations/updates [repeated 4x across cluster]


DEBUG flwr 2023-09-04 07:12:59,005 | fl_server.py:241 | fit_round 15 received 5 results and 0 failures


Server side aggregated 5 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 07:14:30,469 | fl_server.py:112 | fit progress: (15, 2.741219963031423, {'auc': 0.8699248120300752}, 2754.8800091999874)
INFO flwr 2023-09-04 07:14:30,471 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 07:14:30,472 | fl_server.py:140 | FL finished in 2754.883415899996
INFO flwr 2023-09-04 07:14:30,473 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 07:14:30,474 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 07:14:30,476 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 07:14:30,477 | app.py:228 | app_fit: losses_centralized [(0, 110.3438077634011), (1, 5.029574861367838), (2, 6.460258780036969), (3, 5.4288354898336415), (4, 7.54898336414048), (5, 6.576709796672828), (6, 5.857670979667283), (7, 2.478743068391867), (8, 5.101663585951941), (9, 6.706099815157116), (10, 5.717190388170056), (11, 4.624768946395564), (12, 5.635859519408503), (13, 4.8317929759704255), (14

Evaluation on the server: test_mse=2.7412, test_auc=0.8699
History (loss, centralized):
	round 0: 110.3438077634011
	round 1: 5.029574861367838
	round 2: 6.460258780036969
	round 3: 5.4288354898336415
	round 4: 7.54898336414048
	round 5: 6.576709796672828
	round 6: 5.857670979667283
	round 7: 2.478743068391867
	round 8: 5.101663585951941
	round 9: 6.706099815157116
	round 10: 5.717190388170056
	round 11: 4.624768946395564
	round 12: 5.635859519408503
	round 13: 4.8317929759704255
	round 14: 6.645101663585952
	round 15: 2.741219963031423
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.7818865345181134), (2, 0.7939986329460014), (3, 0.8062611073137389), (4, 0.7683595352016405), (5, 0.7872795625427205), (6, 0.8222829801777171), (7, 0.8837320574162678), (8, 0.7914080656185919), (9, 0.8209295967190703), (10, 0.801825017088175), (11, 0.8260150375939849), (12, 0.8368284347231716), (13, 0.7947983595352016), (14, 0.8251537935748462), (15, 0.8699248120300752)]}
[(0, 110.3438077634011), 

In [4]:
split = [0.8, 0.03, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]
do_fl(split)

WARNING flwr 2023-09-04 09:28:18,628 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 09:28:18,630 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


Fold 1:
torch.Size([4500, 22])
torch.Size([120, 22])
torch.Size([120, 22])
torch.Size([120, 22])
torch.Size([120, 22])
torch.Size([120, 22])
torch.Size([120, 22])
torch.Size([100, 22])
torch.Size([100, 22])
torch.Size([100, 22])
Data partitioned across 10 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 10 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.


2023-09-04 09:28:27,922	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 09:28:35,513 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 1739948852.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 869974425.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-09-04 09:28:35,515 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 09:28:35,517 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 09:28:35,518 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=16204) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=16204) 
(launch_and_get_parameters pid=16204) Stack (most recent call first):
(launch_and_get_parameters pid=16204)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=16204)   File "D:\FD\venv\lib\site-packages\torch

(launch_and_get_parameters pid=16204) multiclass client
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 09:29:09,186 | fl_server.py:87 | initial parameters (loss, other metrics): 69.10720887245841, {'auc': 0.5}
INFO flwr 2023-09-04 09:29:09,188 | fl_server.py:97 | FL starting
DEBUG flwr 2023-09-04 09:29:09,189 | fl_server.py:226 | fit_round 1: strategy sampled 5 clients (out of 10)


...

Evaluation on the server: test_mse=7.0481, test_auc=0.7738
Configuring round 15
(launch_and_fit pid=27044) multiclass client
(launch_and_fit pid=28576) Client 2 aggregated_trees length 5
(launch_and_fit pid=28576) Client 2: recieved 5 trees
(launch_and_fit pid=28008) multiclass client [repeated 4x across cluster]
(launch_and_fit pid=16204) Client 0 aggregated_trees length 5 [repeated 4x across cluster]
(launch_and_fit pid=16204) Client 0: recieved 5 trees [repeated 4x across cluster]
(launch_and_fit pid=28008) Client 8: training for 2 iterations/updates
(launch_and_fit pid=16204) Client 0: training for 2 iterations/updates [repeated 4x across cluster]


DEBUG flwr 2023-09-04 10:11:57,693 | fl_server.py:241 | fit_round 15 received 5 results and 0 failures


Server side aggregated 5 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 10:13:50,462 | fl_server.py:112 | fit progress: (15, 3.9741219963031424, {'auc': 0.87796992481203}, 2681.2738501999993)
INFO flwr 2023-09-04 10:13:50,464 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 10:13:50,465 | fl_server.py:140 | FL finished in 2681.277104299981
INFO flwr 2023-09-04 10:13:50,468 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 10:13:50,470 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 10:13:50,471 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 10:13:50,472 | app.py:228 | app_fit: losses_centralized [(0, 69.10720887245841), (1, 8.512014787430683), (2, 9.301293900184843), (3, 7.421441774491682), (4, 9.966728280961183), (5, 7.946395563770795), (6, 6.22920517560074), (7, 9.356746765249538), (8, 9.913123844731977), (9, 7.378927911275416), (10, 7.438077634011091), (11, 4.380776340110906), (12, 8.380776340110906), (13, 6.467652495378928), (14, 7

Evaluation on the server: test_mse=3.9741, test_auc=0.8780
History (loss, centralized):
	round 0: 69.10720887245841
	round 1: 8.512014787430683
	round 2: 9.301293900184843
	round 3: 7.421441774491682
	round 4: 9.966728280961183
	round 5: 7.946395563770795
	round 6: 6.22920517560074
	round 7: 9.356746765249538
	round 8: 9.913123844731977
	round 9: 7.378927911275416
	round 10: 7.438077634011091
	round 11: 4.380776340110906
	round 12: 8.380776340110906
	round 13: 6.467652495378928
	round 14: 7.048059149722736
	round 15: 3.9741219963031424
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.6605878332194122), (2, 0.7620095693779904), (3, 0.7527956254272044), (4, 0.7456801093643199), (5, 0.7705126452494873), (6, 0.792611073137389), (7, 0.7068557758031443), (8, 0.7608680792891319), (9, 0.7587764866712234), (10, 0.7363157894736841), (11, 0.8659193438140806), (12, 0.794771018455229), (13, 0.7779699248120301), (14, 0.7737525632262474), (15, 0.87796992481203)]}
[(0, 69.10720887245841), (1, 

In [5]:
from sklearn.model_selection import StratifiedKFold

def do_fl (split, X=X, y=y):
    start = datetime.datetime.now()

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    flag = 1

    for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X, y)):
        print(f"Fold {fold_idx + 1}:")
        if flag == 2:
            break
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        class_1_indices = torch.where(y_train == 1)[0]
        class_17_indices = torch.where(y_train == 17)[0]
        class_18_indices = torch.where(y_train== 18)[0]
        class_19_indices = torch.where(y_train== 19)[0]
        class_3_indices = torch.where(y_train== 3)[0]
        class_16_indices = torch.where(y_train== 16)[0]
        class_2_indices = torch.where(y_train==2)[0]
        class_0_indices = torch.where(y_train==0)[0]

        class_1_samples = X_train[class_1_indices]
        class_17_samples = X_train[class_17_indices]
        class_18_samples = X_train[class_18_indices]
        class_19_samples = X_train[class_19_indices]
        class_3_samples = X_train[class_3_indices]
        class_16_samples = X_train[class_16_indices]
        class_2_samples = X_train[class_2_indices]
        class_0_samples = X_train[class_0_indices]

        count_1= (50 - class_1_samples.size()[0]) // class_1_samples.size()[0] + 1
        for i in range(count_1):
            X_train = torch.cat((X_train, class_1_samples), dim=0)
            y_train = torch.cat((y_train, torch.full((class_1_samples.size()[0],), 1)), dim=0)

        count_17= (50 - class_17_samples.size()[0]) // class_17_samples.size()[0] + 1
        for i in range(count_17):
            X_train = torch.cat((X_train, class_17_samples), dim=0)
            y_train = torch.cat((y_train, torch.full((class_17_samples.size()[0], ), 17)), dim=0)

        count_18= (50 - class_18_samples.size()[0]) // class_18_samples.size()[0] + 1
        for i in range(count_18):
            X_train = torch.cat((X_train, class_18_samples), dim=0)
            y_train = torch.cat((y_train, torch.full((class_18_samples.size()[0], ), 18)), dim=0)

        count_19= (50 - class_19_samples.size()[0]) // class_19_samples.size()[0] + 1
        for i in range(count_19):
            X_train = torch.cat((X_train, class_19_samples), dim=0)
            y_train = torch.cat((y_train, torch.full((class_19_samples.size()[0], ), 19)), dim=0)

        count_3= (50 - class_3_samples.size()[0]) // class_3_samples.size()[0] + 1
        for i in range(count_3):
            X_train = torch.cat((X_train, class_3_samples), dim=0)
            y_train = torch.cat((y_train, torch.full((class_3_samples.size()[0], ), 3)), dim=0)

        count_16= (50 - class_16_samples.size()[0]) // class_16_samples.size()[0] + 1
        for i in range(count_16):
            X_train = torch.cat((X_train, class_16_samples), dim=0)
            y_train = torch.cat((y_train, torch.full((class_16_samples.size()[0], ), 16)), dim=0)

        count_2= (50 - class_2_samples.size()[0]) // class_2_samples.size()[0] + 1
        for i in range(count_2):
            X_train = torch.cat((X_train, class_2_samples), dim=0)
            y_train = torch.cat((y_train, torch.full((class_2_samples.size()[0], ), 2)), dim=0)

        count_0= (50 - class_0_samples.size()[0]) // class_0_samples.size()[0] + 1
        for i in range(count_0):
            X_train = torch.cat((X_train, class_0_samples), dim=0)
            y_train = torch.cat((y_train, torch.full((class_0_samples.size()[0], ), 0)), dim=0)

        class_4_indices = torch.where(y_train==4)[0]
        class_4_samples = X_train[class_4_indices]

        shuffled_indices = torch.randperm(class_4_samples.shape[0])
        class_4_samples = class_4_samples[shuffled_indices]

        for n in range(5):
            X_train = torch.cat((X_train, class_4_samples[n].unsqueeze(0)), dim=0)

        y_train = torch.cat((y_train, torch.full((5, ), 4)), dim=0)


        trainset = TreeDataset(np.array(X_train, copy=True), np.array(y_train, copy=True))
        testset = TreeDataset(np.array(X_test, copy=True), np.array(y_test, copy=True))

        start = datetime.datetime.now()
        start_experiment(
            split=split,
            trainset=trainset,
            testset=testset,
            num_rounds=15, # 8
            client_tree_num=40,
            client_pool_size=len(split),
            num_iterations=4, #6
            batch_size=2000,
            fraction_fit=0.5, #1
            min_fit_clients=2,
            val_ratio=0.0,
            client_num=len(split),
            task_type = 'multiclass',
            lr=0.001, # was 0.001
            colsample_bytree=1,
            subsample=1,
            max_depth=6,
            min_child_weight=1,
        )

        flag += 1

    end = datetime.datetime.now()
    time = end - start

    print('split', split)
    print('Training time', time)

In [6]:
split = [0.5, 0.5]
do_fl(split)

WARNING flwr 2023-09-04 11:36:31,489 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 11:36:31,573 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


Fold 1:
torch.Size([2700, 22])
torch.Size([2920, 22])
Data partitioned across 2 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 2 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.


(launch_and_fit pid=27044)  [repeated 7x across cluster]
(launch_and_fit pid=27044)   File "<frozen importlib._bootstrap>", line 241 in _call_with_frames_removed [repeated 57x across cluster]
(launch_and_fit pid=27044)   File "<frozen importlib._bootstrap_external>", line 883 in exec_module [repeated 43x across cluster]
(launch_and_fit pid=27044)   File "<frozen importlib._bootstrap>", line 688 in _load_unlocked [repeated 44x across cluster]
(launch_and_fit pid=27044)   File "<frozen importlib._bootstrap>", line 1006 in _find_and_load_unlocked [repeated 51x across cluster]
(launch_and_fit pid=27044)   File "<frozen importlib._bootstrap>", line 1027 in _find_and_load [repeated 51x across cluster]
(launch_and_fit pid=27044)   File "D:\FD\venv\lib\site-packages\torchmetrics\functional\__init__.py", line 14 in <module> [repeated 3x across cluster]
(launch_and_fit pid=27044)   File "<frozen importlib._bootstrap>", line 1078 in _handle_fromlist [repeated 6x across cluster]
(launch_and_fit pi

(launch_and_get_parameters pid=17024) multiclass client


INFO flwr 2023-09-04 11:37:17,845 | fl_server.py:292 | Received initial parameters from one random client
INFO flwr 2023-09-04 11:37:17,848 | fl_server.py:84 | Evaluating initial parameters


...

Evaluation on the server: test_mse=1.8189, test_auc=0.9059
Configuring round 15
(launch_and_fit pid=17024) multiclass client
(launch_and_fit pid=17024) multiclass client
(launch_and_fit pid=17024) Client 0 aggregated_trees length 2
(launch_and_fit pid=17024) Client 0: recieved 2 trees
(launch_and_fit pid=23548) Client 0: recieved 2 trees
(launch_and_fit pid=17024) Client 0: training for 4 iterations/updates
(launch_and_fit pid=23548) Client 0: training for 4 iterations/updates
(launch_and_fit pid=23548) Client 0: training for 4 iterations/updates


DEBUG flwr 2023-09-04 12:05:52,827 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 12:06:16,859 | fl_server.py:112 | fit progress: (15, 1.7282809611829943, {'auc': 0.9039166097060833}, 1715.1145360000082)
INFO flwr 2023-09-04 12:06:16,862 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 12:06:16,864 | fl_server.py:140 | FL finished in 1715.1189324000152
INFO flwr 2023-09-04 12:06:16,865 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 12:06:16,867 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 12:06:16,869 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 12:06:16,870 | app.py:228 | app_fit: losses_centralized [(0, 69.31238447319778), (1, 2.367837338262477), (2, 2.057301293900185), (3, 1.8761552680221811), (4, 1.7837338262476894), (5, 1.731977818853974), (6, 1.8003696857670979), (7, 1.756007393715342), (8, 1.7024029574861368), (9, 1.7116451016635859), (10, 1.7486136783733826), (11, 1.8354898336414047), (12, 1.7744916820702403), (13, 1.826247689463

Evaluation on the server: test_mse=1.7283, test_auc=0.9039
History (loss, centralized):
	round 0: 69.31238447319778
	round 1: 2.367837338262477
	round 2: 2.057301293900185
	round 3: 1.8761552680221811
	round 4: 1.7837338262476894
	round 5: 1.731977818853974
	round 6: 1.8003696857670979
	round 7: 1.756007393715342
	round 8: 1.7024029574861368
	round 9: 1.7116451016635859
	round 10: 1.7486136783733826
	round 11: 1.8354898336414047
	round 12: 1.7744916820702403
	round 13: 1.8262476894639557
	round 14: 1.8188539741219962
	round 15: 1.7282809611829943
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8576965140123035), (2, 0.8965823650034176), (3, 0.9020369104579631), (4, 0.9151948051948052), (5, 0.9169514695830485), (6, 0.8997881066302118), (7, 0.9041626794258373), (8, 0.9057963089542037), (9, 0.9079835953520164), (10, 0.9041011619958988), (11, 0.9077375256322625), (12, 0.9005878332194122), (13, 0.9122966507177035), (14, 0.9058578263841421), (15, 0.9039166097060833)]}
[(0, 69.312384

In [7]:
split = [0.6, 0.4]
do_fl(split)

WARNING flwr 2023-09-04 12:06:17,121 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 12:06:17,123 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


Fold 1:
torch.Size([3300, 22])
torch.Size([2320, 22])
Data partitioned across 2 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 2 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.
(launch_and_fit pid=23548) Client 1: training for 4 iterations/updates


2023-09-04 12:06:28,244	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 12:06:40,592 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 1740300288.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 870150144.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-09-04 12:06:40,594 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 12:06:40,597 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 12:06:40,599 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=26964) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=26964) 
(launch_and_get_parameters pid=26964) Stack (most recent call first):
(launch_and_get_parameters pid=26964)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=26964)   File "D:\FD\venv\lib\site-packages\torch

(launch_and_get_parameters pid=26964) multiclass client


INFO flwr 2023-09-04 12:06:55,706 | fl_server.py:292 | Received initial parameters from one random client
INFO flwr 2023-09-04 12:06:55,708 | fl_server.py:84 | Evaluating initial parameters


...

Evaluation on the server: test_mse=1.7301, test_auc=0.9169
Configuring round 15
(launch_and_fit pid=26964) multiclass client
(launch_and_fit pid=19596) Client 1 aggregated_trees length 2
(launch_and_fit pid=19596) Client 1: recieved 2 trees
(launch_and_fit pid=19596) Client 1: recieved 2 trees
(launch_and_fit pid=26964) Client 0: recieved 2 trees
(launch_and_fit pid=26964) Client 0: recieved 2 trees
(launch_and_fit pid=19596) Client 1: training for 4 iterations/updates
(launch_and_fit pid=26964) Client 0: training for 4 iterations/updates


DEBUG flwr 2023-09-04 12:36:12,670 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 12:36:37,596 | fl_server.py:112 | fit progress: (15, 1.8151571164510167, {'auc': 0.9113123718386876}, 1764.0100485000003)
INFO flwr 2023-09-04 12:36:37,599 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 12:36:37,601 | fl_server.py:140 | FL finished in 1764.0152855000051
INFO flwr 2023-09-04 12:36:37,603 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 12:36:37,609 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 12:36:37,612 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 12:36:37,615 | app.py:228 | app_fit: losses_centralized [(0, 69.090573012939), (1, 2.4140480591497226), (2, 2.048059149722736), (3, 2.014787430683919), (4, 2.009242144177449), (5, 1.9038817005545285), (6, 1.8243992606284658), (7, 1.9500924214417745), (8, 1.844731977818854), (9, 1.8170055452865064), (10, 1.9038817005545285), (11, 1.8724584103512014), (12, 1.700554528650647), (13, 1.809611829944547

Evaluation on the server: test_mse=1.8152, test_auc=0.9113
History (loss, centralized):
	round 0: 69.090573012939
	round 1: 2.4140480591497226
	round 2: 2.048059149722736
	round 3: 2.014787430683919
	round 4: 2.009242144177449
	round 5: 1.9038817005545285
	round 6: 1.8243992606284658
	round 7: 1.9500924214417745
	round 8: 1.844731977818854
	round 9: 1.8170055452865064
	round 10: 1.9038817005545285
	round 11: 1.8724584103512014
	round 12: 1.700554528650647
	round 13: 1.8096118299445472
	round 14: 1.7301293900184842
	round 15: 1.8151571164510167
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8582501708817498), (2, 0.8904511278195488), (3, 0.9063499658236501), (4, 0.9134996582365003), (5, 0.917259056732741), (6, 0.92107997265892), (7, 0.9192002734107997), (8, 0.9205263157894736), (9, 0.9224675324675325), (10, 0.9165208475734792), (11, 0.9225905673274094), (12, 0.9186466165413534), (13, 0.9225290498974709), (14, 0.91688995215311), (15, 0.9113123718386876)]}
[(0, 69.090573012939),

In [8]:
split = [0.8, 0.2]
do_fl(split)

Fold 1:


WARNING flwr 2023-09-04 12:36:37,846 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 12:36:37,849 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


torch.Size([4500, 22])
torch.Size([1120, 22])
Data partitioned across 2 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 2 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.


2023-09-04 12:36:49,508	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 12:37:01,015 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 1571006055.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 785503027.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-09-04 12:37:01,017 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 12:37:01,020 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 12:37:01,022 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=22880) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=22880) 
(launch_and_get_parameters pid=22880) Stack (most recent call first):
(launch_and_get_parameters pid=22880)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=22880)   File "D:\FD\venv\lib\site-packages\torch

(launch_and_get_parameters pid=22880) multiclass client


...

Evaluation on the server: test_mse=1.7172, test_auc=0.9188
Configuring round 15
(launch_and_fit pid=22880) multiclass client
(launch_and_fit pid=22880) Client 1 aggregated_trees length 2
(launch_and_fit pid=22880) Client 1: recieved 2 trees
(launch_and_fit pid=18084) Client 0: recieved 2 trees
(launch_and_fit pid=18084) Client 0: recieved 2 trees
(launch_and_fit pid=18084) Client 0: recieved 2 trees
(launch_and_fit pid=22880) Client 1: training for 4 iterations/updates
(launch_and_fit pid=18084) Client 0: training for 4 iterations/updates


DEBUG flwr 2023-09-04 13:08:08,932 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 13:08:32,508 | fl_server.py:112 | fit progress: (15, 1.7615526802218116, {'auc': 0.9205878332194123}, 1848.769048200018)
INFO flwr 2023-09-04 13:08:32,510 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 13:08:32,512 | fl_server.py:140 | FL finished in 1848.773690800008
INFO flwr 2023-09-04 13:08:32,514 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 13:08:32,517 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 13:08:32,519 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 13:08:32,521 | app.py:228 | app_fit: losses_centralized [(0, 84.90388170055454), (1, 2.3752310536044363), (2, 2.4824399260628467), (3, 2.1256931608133085), (4, 2.024029574861368), (5, 2.0702402957486137), (6, 2.0166358595194085), (7, 2.1072088724584104), (8, 1.988909426987061), (9, 1.8576709796672828), (10, 1.88724584103512), (11, 1.8022181146025877), (12, 1.8354898336414047), (13, 1.77264325323475

Evaluation on the server: test_mse=1.7616, test_auc=0.9206
History (loss, centralized):
	round 0: 84.90388170055454
	round 1: 2.3752310536044363
	round 2: 2.4824399260628467
	round 3: 2.1256931608133085
	round 4: 2.024029574861368
	round 5: 2.0702402957486137
	round 6: 2.0166358595194085
	round 7: 2.1072088724584104
	round 8: 1.988909426987061
	round 9: 1.8576709796672828
	round 10: 1.88724584103512
	round 11: 1.8022181146025877
	round 12: 1.8354898336414047
	round 13: 1.7726432532347505
	round 14: 1.7171903881700554
	round 15: 1.7615526802218116
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8838550922761449), (2, 0.8896787423103212), (3, 0.90215994531784), (4, 0.9022214627477786), (5, 0.9074914559125083), (6, 0.9037935748462065), (7, 0.9114969241285031), (8, 0.9114354066985646), (9, 0.9112508544087491), (10, 0.9114354066985646), (11, 0.9169514695830485), (12, 0.9132535885167463), (13, 0.917012987012987), (14, 0.9187696514012303), (15, 0.9205878332194123)]}
[(0, 84.903881700

In [9]:
split = [0.34, 0.33, 0.33]
do_fl(split)

Fold 1:


WARNING flwr 2023-09-04 13:08:32,727 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 13:08:32,729 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


torch.Size([1740, 22])
torch.Size([1920, 22])
torch.Size([1860, 22])
Data partitioned across 3 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 3 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.


2023-09-04 13:08:45,965	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 13:08:59,994 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 1463220635.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 731610316.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-09-04 13:08:59,997 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 13:08:59,999 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 13:09:00,001 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=29672) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=29672) 
(launch_and_get_parameters pid=29672) Stack (most recent call first):
(launch_and_get_parameters pid=29672)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=29672)   File "D:\FD\venv\lib\site-packages\torch

(launch_and_get_parameters pid=29672) multiclass client


...

Evaluation on the server: test_mse=2.8152, test_auc=0.8917
Configuring round 15
(launch_and_fit pid=29672) multiclass client
(launch_and_fit pid=29672) multiclass client
(launch_and_fit pid=27532) Client 0 aggregated_trees length 2
(launch_and_fit pid=27532) Client 0: recieved 2 trees
(launch_and_fit pid=27532) Client 0: recieved 2 trees
(launch_and_fit pid=27532) Client 0: training for 4 iterations/updates
(launch_and_fit pid=29672) Client 0: training for 4 iterations/updates
(launch_and_fit pid=29672) Client 0: training for 4 iterations/updates


DEBUG flwr 2023-09-04 13:55:13,350 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 13:55:44,874 | fl_server.py:112 | fit progress: (15, 2.698706099815157, {'auc': 0.8707587149692413}, 2763.708749000012)
INFO flwr 2023-09-04 13:55:44,877 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 13:55:44,879 | fl_server.py:140 | FL finished in 2763.7131068000162
INFO flwr 2023-09-04 13:55:44,880 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 13:55:44,883 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 13:55:44,885 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 13:55:44,887 | app.py:228 | app_fit: losses_centralized [(0, 124.14417744916821), (1, 2.5064695009242146), (2, 3.22365988909427), (3, 2.5471349353049906), (4, 2.7393715341959335), (5, 2.6580406654343807), (6, 2.6395563770794825), (7, 2.5859519408502774), (8, 2.857670979667283), (9, 2.746765249537893), (10, 2.656192236598891), (11, 2.756007393715342), (12, 2.3493530499075788), (13, 2.602587800369686

Evaluation on the server: test_mse=2.6987, test_auc=0.8708
History (loss, centralized):
	round 0: 124.14417744916821
	round 1: 2.5064695009242146
	round 2: 3.22365988909427
	round 3: 2.5471349353049906
	round 4: 2.7393715341959335
	round 5: 2.6580406654343807
	round 6: 2.6395563770794825
	round 7: 2.5859519408502774
	round 8: 2.857670979667283
	round 9: 2.746765249537893
	round 10: 2.656192236598891
	round 11: 2.756007393715342
	round 12: 2.3493530499075788
	round 13: 2.602587800369686
	round 14: 2.8151571164510165
	round 15: 2.698706099815157
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8625632262474369), (2, 0.8721120984278878), (3, 0.883485987696514), (4, 0.8765208475734791), (5, 0.883609022556391), (6, 0.8761517429938482), (7, 0.8777238550922761), (8, 0.8811414900888586), (9, 0.864688995215311), (10, 0.8812987012987012), (11, 0.8876144907723855), (12, 0.8971360218728639), (13, 0.8829938482570062), (14, 0.8916814764183184), (15, 0.8707587149692413)]}
[(0, 124.14417744916

In [10]:
split = [0.6, 0.2, 0.2]
do_fl(split)

Fold 1:
torch.Size([3300, 22])
torch.Size([1200, 22])
torch.Size([1120, 22])
Data partitioned across 3 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 3 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.


WARNING flwr 2023-09-04 13:55:45,091 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 13:55:45,094 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


(launch_and_fit pid=29672) Client 1: training for 4 iterations/updates


2023-09-04 13:55:55,368	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 13:56:06,420 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 1371377664.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 685688832.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-09-04 13:56:06,423 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 13:56:06,425 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 13:56:06,427 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=16048) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=16048) 
(launch_and_get_parameters pid=16048) Stack (most recent call first):
(launch_and_get_parameters pid=16048)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=16048)   File "D:\FD\venv\lib\site-packages\torch

(launch_and_get_parameters pid=16048) multiclass client


...

Evaluation on the server: test_mse=3.0869, test_auc=0.8742
Configuring round 15
(launch_and_fit pid=16048) multiclass client
(launch_and_fit pid=16048) multiclass client
(launch_and_fit pid=16048) Client 2 aggregated_trees length 2
(launch_and_fit pid=16048) Client 2: recieved 2 trees
(launch_and_fit pid=16048) Client 2: training for 4 iterations/updates
(launch_and_fit pid=25188) Client 2: training for 4 iterations/updates
(launch_and_fit pid=25188) Client 2: training for 4 iterations/updates
(launch_and_fit pid=25188) Client 2: training for 4 iterations/updates


DEBUG flwr 2023-09-04 14:30:55,257 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 14:31:19,603 | fl_server.py:112 | fit progress: (15, 3.77818853974122, {'auc': 0.8297744360902255}, 2055.0228719000006)
INFO flwr 2023-09-04 14:31:19,605 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 14:31:19,607 | fl_server.py:140 | FL finished in 2055.027468400018
INFO flwr 2023-09-04 14:31:19,609 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 14:31:19,611 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 14:31:19,613 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 14:31:19,615 | app.py:228 | app_fit: losses_centralized [(0, 76.31423290203327), (1, 3.5508317929759703), (2, 2.3512014787430684), (3, 2.3696857670979665), (4, 2.933456561922366), (5, 4.238447319778189), (6, 2.707948243992606), (7, 4.036968576709796), (8, 3.4621072088724585), (9, 2.955637707948244), (10, 2.0258780036968576), (11, 2.789279112754159), (12, 2.598890942698706), (13, 2.201478743068392), 

Evaluation on the server: test_mse=3.7782, test_auc=0.8298
History (loss, centralized):
	round 0: 76.31423290203327
	round 1: 3.5508317929759703
	round 2: 2.3512014787430684
	round 3: 2.3696857670979665
	round 4: 2.933456561922366
	round 5: 4.238447319778189
	round 6: 2.707948243992606
	round 7: 4.036968576709796
	round 8: 3.4621072088724585
	round 9: 2.955637707948244
	round 10: 2.0258780036968576
	round 11: 2.789279112754159
	round 12: 2.598890942698706
	round 13: 2.201478743068392
	round 14: 3.086876155268022
	round 15: 3.77818853974122
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.874025974025974), (2, 0.8914969241285031), (3, 0.899542036910458), (4, 0.8802802460697198), (5, 0.8317156527682845), (6, 0.8625905673274094), (7, 0.8210252904989748), (8, 0.8615447710184553), (9, 0.868045112781955), (10, 0.8975393028024606), (11, 0.8651811346548188), (12, 0.870082023239918), (13, 0.8863841421736158), (14, 0.8742105263157894), (15, 0.8297744360902255)]}
[(0, 76.31423290203327), 

In [11]:
split = [0.8, 0.1, 0.1]
do_fl(split)

Fold 1:
torch.Size([4500, 22])
torch.Size([600, 22])
torch.Size([520, 22])


WARNING flwr 2023-09-04 14:31:19,820 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 14:31:19,823 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


Data partitioned across 3 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 3 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.
(launch_and_fit pid=25188) Client 1: training for 4 iterations/updates


(launch_and_fit pid=25188)   File "<frozen importlib._bootstrap>", line 241 in _call_with_frames_removed [repeated 4x across cluster]
(launch_and_fit pid=25188)   File "<frozen importlib._bootstrap_external>", line 883 in exec_module [repeated 3x across cluster]
(launch_and_fit pid=25188)   File "<frozen importlib._bootstrap>", line 688 in _load_unlocked [repeated 3x across cluster]
(launch_and_fit pid=25188)   File "<frozen importlib._bootstrap>", line 1006 in _find_and_load_unlocked [repeated 4x across cluster]
(launch_and_fit pid=25188)   File "<frozen importlib._bootstrap>", line 1027 in _find_and_load [repeated 4x across cluster]
2023-09-04 14:31:29,786	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 14:31:41,740 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 1045079655.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 522539827.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-09-04 14:31:41,743 | fl_server.py:78 | fit num_rounds 15
INFO fl

(launch_and_get_parameters pid=25592) multiclass client


...

Evaluation on the server: test_mse=4.5823, test_auc=0.8286
Configuring round 15
(launch_and_fit pid=25592) multiclass client
(launch_and_fit pid=25592) Client 1 aggregated_trees length 2
(launch_and_fit pid=25592) Client 1: recieved 2 trees
(launch_and_fit pid=22268) Client 0: recieved 2 trees
(launch_and_fit pid=22268) Client 0: recieved 2 trees
(launch_and_fit pid=22268) Client 0: recieved 2 trees
(launch_and_fit pid=25592) Client 1: training for 4 iterations/updates
(launch_and_fit pid=22268) Client 0: training for 4 iterations/updates


DEBUG flwr 2023-09-04 15:01:37,562 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 15:02:20,720 | fl_server.py:112 | fit progress: (15, 2.2495378927911274, {'auc': 0.8968284347231716}, 1808.8163486999983)
INFO flwr 2023-09-04 15:02:20,724 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 15:02:20,726 | fl_server.py:140 | FL finished in 1808.8217996000021
INFO flwr 2023-09-04 15:02:20,728 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 15:02:20,730 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 15:02:20,733 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 15:02:20,737 | app.py:228 | app_fit: losses_centralized [(0, 94.28465804066543), (1, 5.33086876155268), (2, 2.3031423290203326), (3, 4.990757855822551), (4, 3.088724584103512), (5, 3.876155268022181), (6, 5.345656192236599), (7, 2.5249537892791127), (8, 1.9390018484288354), (9, 1.8927911275415896), (10, 1.8096118299445472), (11, 5.2772643253234754), (12, 3.7301293900184844), (13, 3.96857670979667

Evaluation on the server: test_mse=2.2495, test_auc=0.8968
History (loss, centralized):
	round 0: 94.28465804066543
	round 1: 5.33086876155268
	round 2: 2.3031423290203326
	round 3: 4.990757855822551
	round 4: 3.088724584103512
	round 5: 3.876155268022181
	round 6: 5.345656192236599
	round 7: 2.5249537892791127
	round 8: 1.9390018484288354
	round 9: 1.8927911275415896
	round 10: 1.8096118299445472
	round 11: 5.2772643253234754
	round 12: 3.7301293900184844
	round 13: 3.9685767097966727
	round 14: 4.582255083179297
	round 15: 2.2495378927911274
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8202529049897471), (2, 0.8786466165413533), (3, 0.8282023239917977), (4, 0.8516814764183185), (5, 0.8423786739576212), (6, 0.8094053315105948), (7, 0.897012987012987), (8, 0.8923308270676691), (9, 0.9059193438140807), (10, 0.9075529733424471), (11, 0.8029938482570061), (12, 0.8149829118250171), (13, 0.8555365686944634), (14, 0.8286329460013669), (15, 0.8968284347231716)]}
[(0, 94.2846580406

In [12]:
split = [0.2, 0.2, 0.2, 0.2, 0.2]
do_fl(split)

Fold 1:

WARNING flwr 2023-09-04 15:02:21,025 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 15:02:21,028 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)



torch.Size([1000, 22])
torch.Size([1100, 22])
torch.Size([1200, 22])
torch.Size([1200, 22])
torch.Size([1120, 22])
Data partitioned across 5 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 5 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.


2023-09-04 15:02:30,050	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 15:02:40,029 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 1911317300.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 955658649.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-09-04 15:02:40,033 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 15:02:40,036 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 15:02:40,038 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=23640) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=23640) 
(launch_and_get_parameters pid=23640) Stack (most recent call first):
(launch_and_get_parameters pid=23640)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=23640)   File "D:\FD\venv\lib\site-packages\torch

(launch_and_get_parameters pid=23640) multiclass client


...

Evaluation on the server: test_mse=2.8447, test_auc=0.8633
Configuring round 15
(launch_and_fit pid=23640) multiclass client
(launch_and_fit pid=23640) multiclass client
(launch_and_fit pid=24612) Client 0 aggregated_trees length 2
(launch_and_fit pid=24612) Client 0: recieved 2 trees
(launch_and_fit pid=24612) Client 0: training for 4 iterations/updates
(launch_and_fit pid=24612) Client 0: training for 4 iterations/updates
(launch_and_fit pid=23640) Client 0: training for 4 iterations/updates
(launch_and_fit pid=23640) Client 0: training for 4 iterations/updates


DEBUG flwr 2023-09-04 15:47:19,970 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 15:48:04,419 | fl_server.py:112 | fit progress: (15, 3.2698706099815156, {'auc': 0.8626247436773753}, 2661.398338400002)
INFO flwr 2023-09-04 15:48:04,422 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 15:48:04,424 | fl_server.py:140 | FL finished in 2661.4034975999966
INFO flwr 2023-09-04 15:48:04,426 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 15:48:04,429 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 15:48:04,431 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 15:48:04,434 | app.py:228 | app_fit: losses_centralized [(0, 72.3438077634011), (1, 18.182994454713494), (2, 3.343807763401109), (3, 3.5268022181146024), (4, 3.388170055452865), (5, 3.8262476894639557), (6, 2.7486136783733826), (7, 3.033271719038817), (8, 3.4602587800369684), (9, 3.166358595194085), (10, 3.1312384473197783), (11, 3.099815157116451), (12, 2.9944547134935307), (13, 3.231053604436229

Evaluation on the server: test_mse=3.2699, test_auc=0.8626
History (loss, centralized):
	round 0: 72.3438077634011
	round 1: 18.182994454713494
	round 2: 3.343807763401109
	round 3: 3.5268022181146024
	round 4: 3.388170055452865
	round 5: 3.8262476894639557
	round 6: 2.7486136783733826
	round 7: 3.033271719038817
	round 8: 3.4602587800369684
	round 9: 3.166358595194085
	round 10: 3.1312384473197783
	round 11: 3.099815157116451
	round 12: 2.9944547134935307
	round 13: 3.2310536044362292
	round 14: 2.844731977818854
	round 15: 3.2698706099815156
History (metrics, centralized):
{'auc': [(0, 0.49799726589200277), (1, 0.4808680792891319), (2, 0.8516541353383459), (3, 0.8665071770334927), (4, 0.8462611073137389), (5, 0.8481408065618592), (6, 0.8868762816131237), (7, 0.8608065618591935), (8, 0.8667532467532468), (9, 0.8838550922761449), (10, 0.8749487354750513), (11, 0.8861654135338344), (12, 0.8895557074504443), (13, 0.860745044429255), (14, 0.8633014354066986), (15, 0.8626247436773753)]}
[(

In [13]:
split = [0.6, 0.1, 0.1, 0.1, 0.1]
do_fl(split)

Fold 1:


WARNING flwr 2023-09-04 15:48:04,683 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 15:48:04,685 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


torch.Size([3300, 22])
torch.Size([600, 22])
torch.Size([600, 22])
torch.Size([600, 22])
torch.Size([520, 22])
Data partitioned across 5 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 5 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.
(launch_and_fit pid=23640) Client 2: training for 4 iterations/updates


2023-09-04 15:48:16,536	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 15:48:25,772 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 1222007195.0, 'node:127.0.0.1': 1.0, 'object_store_memory': 611003596.0, 'GPU': 1.0, 'CPU': 8.0}
INFO flwr 2023-09-04 15:48:25,775 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 15:48:25,776 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 15:48:25,778 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=24476) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=24476) 
(launch_and_get_parameters pid=24476) Stack (most recent call first):
(launch_and_get_parameters pid=24476)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=24476)   File "D:\FD\venv\lib\site-packages\torch

(launch_and_get_parameters pid=24476) multiclass client


...

Evaluation on the server: test_mse=3.5970, test_auc=0.8509
Configuring round 15
(launch_and_fit pid=22180) multiclass client
(launch_and_fit pid=24476) Client 3 aggregated_trees length 2
(launch_and_fit pid=24476) Client 3: recieved 2 trees
(launch_and_fit pid=22180) Client 1: training for 4 iterations/updates
(launch_and_fit pid=24476) Client 1: training for 4 iterations/updates
(launch_and_fit pid=22180) Client 1: training for 4 iterations/updates
(launch_and_fit pid=22180) Client 1: training for 4 iterations/updates


DEBUG flwr 2023-09-04 16:14:43,076 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 16:15:15,143 | fl_server.py:112 | fit progress: (15, 4.011090573012939, {'auc': 0.8403417634996583}, 1565.586194899981)
INFO flwr 2023-09-04 16:15:15,145 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 16:15:15,147 | fl_server.py:140 | FL finished in 1565.5896322999906
INFO flwr 2023-09-04 16:15:15,150 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 16:15:15,152 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 16:15:15,155 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 16:15:15,157 | app.py:228 | app_fit: losses_centralized [(0, 108.6746765249538), (1, 4.530499075785582), (2, 2.608133086876155), (3, 5.56007393715342), (4, 4.499075785582255), (5, 3.9168207024029575), (6, 3.6432532347504623), (7, 4.654343807763401), (8, 3.9260628465804066), (9, 4.800369685767098), (10, 3.837338262476895), (11, 3.981515711645102), (12, 3.798521256931608), (13, 3.866913123844732), (1

Evaluation on the server: test_mse=4.0111, test_auc=0.8403
History (loss, centralized):
	round 0: 108.6746765249538
	round 1: 4.530499075785582
	round 2: 2.608133086876155
	round 3: 5.56007393715342
	round 4: 4.499075785582255
	round 5: 3.9168207024029575
	round 6: 3.6432532347504623
	round 7: 4.654343807763401
	round 8: 3.9260628465804066
	round 9: 4.800369685767098
	round 10: 3.837338262476895
	round 11: 3.981515711645102
	round 12: 3.798521256931608
	round 13: 3.866913123844732
	round 14: 3.597042513863216
	round 15: 4.011090573012939
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8371086807928914), (2, 0.8802802460697198), (3, 0.8107928913192071), (4, 0.8503896103896105), (5, 0.8396035543403964), (6, 0.8600341763499658), (7, 0.8218865345181134), (8, 0.8326110731373889), (9, 0.8244429254955571), (10, 0.8451811346548189), (11, 0.83904989747095), (12, 0.8399111414900888), (13, 0.8416678058783322), (14, 0.8509432672590568), (15, 0.8403417634996583)]}
[(0, 108.6746765249538), 

In [6]:
split = [0.8, 0.05, 0.05, 0.05, 0.05]
do_fl(split)

WARNING flwr 2023-09-04 16:54:52,470 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 16:54:52,514 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


Fold 1:
torch.Size([4500, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([200, 22])
Data partitioned across 5 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 5 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.


2023-09-04 16:54:59,576	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 16:55:05,485 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 1353938535.0, 'object_store_memory': 676969267.0, 'node:127.0.0.1': 1.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-09-04 16:55:05,487 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 16:55:05,488 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 16:55:05,489 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=5816) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=5816) 
(launch_and_get_parameters pid=5816) Stack (most recent call first):
(launch_and_get_parameters pid=5816)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=5816)   File "D:\FD\venv\lib\site-packages\torch\_ops

(launch_and_get_parameters pid=5816) multiclass client


INFO flwr 2023-09-04 16:55:12,434 | fl_server.py:292 | Received initial parameters from one random client
INFO flwr 2023-09-04 16:55:12,435 | fl_server.py:84 | Evaluating initial parameters


...

Evaluation on the server: test_mse=5.8078, test_auc=0.8033
Configuring round 15
(launch_and_fit pid=5816) multiclass client
(launch_and_fit pid=5816) multiclass client
(launch_and_fit pid=5816) Client 1 aggregated_trees length 2
(launch_and_fit pid=5816) Client 1: recieved 2 trees
(launch_and_fit pid=5816) Client 1: training for 4 iterations/updates
(launch_and_fit pid=1404) Client 1: training for 4 iterations/updates
(launch_and_fit pid=1404) Client 1: training for 4 iterations/updates
(launch_and_fit pid=1404) Client 1: training for 4 iterations/updates


DEBUG flwr 2023-09-04 17:18:18,511 | fl_server.py:241 | fit_round 15 received 2 results and 0 failures


Server side aggregated 2 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 17:18:36,224 | fl_server.py:112 | fit progress: (15, 4.898336414048059, {'auc': 0.8125495557074505}, 1392.4282976000104)
INFO flwr 2023-09-04 17:18:36,226 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 17:18:36,228 | fl_server.py:140 | FL finished in 1392.431735100021
INFO flwr 2023-09-04 17:18:36,230 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 17:18:36,232 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 17:18:36,234 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 17:18:36,235 | app.py:228 | app_fit: losses_centralized [(0, 124.14417744916821), (1, 4.251386321626617), (2, 2.733826247689464), (3, 5.796672828096118), (4, 8.434380776340111), (5, 6.382624768946395), (6, 7.046210720887246), (7, 5.340110905730129), (8, 5.648798521256932), (9, 6.487985212569316), (10, 5.506469500924214), (11, 4.733826247689464), (12, 5.197781885397412), (13, 5.166358595194085), (14

Evaluation on the server: test_mse=4.8983, test_auc=0.8125
History (loss, centralized):
	round 0: 124.14417744916821
	round 1: 4.251386321626617
	round 2: 2.733826247689464
	round 3: 5.796672828096118
	round 4: 8.434380776340111
	round 5: 6.382624768946395
	round 6: 7.046210720887246
	round 7: 5.340110905730129
	round 8: 5.648798521256932
	round 9: 6.487985212569316
	round 10: 5.506469500924214
	round 11: 4.733826247689464
	round 12: 5.197781885397412
	round 13: 5.166358595194085
	round 14: 5.8077634011090575
	round 15: 4.898336414048059
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8024401913875597), (2, 0.864935064935065), (3, 0.8066643882433355), (4, 0.7838892686261107), (5, 0.786725905673274), (6, 0.781578947368421), (7, 0.7931032125768968), (8, 0.7904853041695147), (9, 0.7725495557074505), (10, 0.7694395078605606), (11, 0.807587149692413), (12, 0.7900546821599453), (13, 0.8105468215994531), (14, 0.803274094326726), (15, 0.8125495557074505)]}
[(0, 124.14417744916821), (1

In [7]:
split = [0.1, 0.1, 0.1, 0.1, 0.1,0.1, 0.1, 0.1, 0.1, 0.1]
do_fl(split)

Fold 1:
torch.Size([500, 22])
torch.Size([500, 22])
torch.Size([500, 22])
torch.Size([600, 22])
torch.Size([600, 22])
torch.Size([600, 22])
torch.Size([600, 22])
torch.Size([600, 22])
torch.Size([600, 22])
torch.Size([520, 22])
Data partitioned across 10 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 10 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.


WARNING flwr 2023-09-04 17:18:36,409 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 17:18:36,411 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


(launch_and_fit pid=1404) Client 2: training for 4 iterations/updates


2023-09-04 17:18:47,437	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-09-04 17:18:53,971 | app.py:179 | Flower VCE: Ray initialized with resources: {'memory': 1390028391.0, 'object_store_memory': 695014195.0, 'node:127.0.0.1': 1.0, 'CPU': 8.0, 'GPU': 1.0}
INFO flwr 2023-09-04 17:18:53,975 | fl_server.py:78 | fit num_rounds 15
INFO flwr 2023-09-04 17:18:53,977 | fl_server.py:81 | Initializing global parameters
INFO flwr 2023-09-04 17:18:53,979 | fl_server.py:287 | Requesting initial parameters from one random client
(launch_and_get_parameters pid=21396) Windows fatal exception: code 0xc0000139
(launch_and_get_parameters pid=21396) 
(launch_and_get_parameters pid=21396) Stack (most recent call first):
(launch_and_get_parameters pid=21396)   File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.10_3.10.3056.0_x64__qbz5n2kfra8p0\lib\ctypes\__init__.py", line 374 in __init__
(launch_and_get_parameters pid=21396)   File "D:\FD\venv\lib\site-packages\torch

(launch_and_get_parameters pid=21396) multiclass client


...

Evaluation on the server: test_mse=4.0018, test_auc=0.8316
Configuring round 15
(launch_and_fit pid=9328) multiclass client
(launch_and_fit pid=21396) Client 0 aggregated_trees length 5
(launch_and_fit pid=21396) Client 0: recieved 5 trees
(launch_and_fit pid=9328) Client 4: training for 4 iterations/updates
(launch_and_fit pid=30360) multiclass client [repeated 4x across cluster]
(launch_and_fit pid=30360) Client 5 aggregated_trees length 5 [repeated 4x across cluster]
(launch_and_fit pid=30360) Client 5: recieved 5 trees [repeated 4x across cluster]


DEBUG flwr 2023-09-04 18:41:29,314 | fl_server.py:241 | fit_round 15 received 5 results and 0 failures


Server side aggregated 5 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 18:43:04,366 | fl_server.py:112 | fit progress: (15, 3.5859519408502774, {'auc': 0.8629938482570062}, 4974.346492899989)
INFO flwr 2023-09-04 18:43:04,368 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 18:43:04,369 | fl_server.py:140 | FL finished in 4974.350189599994
INFO flwr 2023-09-04 18:43:04,371 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 18:43:04,372 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 18:43:04,375 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 18:43:04,376 | app.py:228 | app_fit: losses_centralized [(0, 85.452865064695), (1, 4.4713493530499075), (2, 4.6802218114602585), (3, 5.578558225508318), (4, 5.166358595194085), (5, 4.382624768946395), (6, 3.789279112754159), (7, 4.44177449168207), (8, 3.922365988909427), (9, 5.349353049907578), (10, 4.3345656192236595), (11, 3.720887245841035), (12, 4.297597042513863), (13, 7.009242144177449), (14,

Evaluation on the server: test_mse=3.5860, test_auc=0.8630
History (loss, centralized):
	round 0: 85.452865064695
	round 1: 4.4713493530499075
	round 2: 4.6802218114602585
	round 3: 5.578558225508318
	round 4: 5.166358595194085
	round 5: 4.382624768946395
	round 6: 3.789279112754159
	round 7: 4.44177449168207
	round 8: 3.922365988909427
	round 9: 5.349353049907578
	round 10: 4.3345656192236595
	round 11: 3.720887245841035
	round 12: 4.297597042513863
	round 13: 7.009242144177449
	round 14: 4.00184842883549
	round 15: 3.5859519408502774
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8319617224880382), (2, 0.8524538619275461), (3, 0.8331647300068352), (4, 0.8310389610389611), (5, 0.8577238550922761), (6, 0.8707587149692413), (7, 0.8455502392344498), (8, 0.8466302118933698), (9, 0.8203144224196857), (10, 0.8542105263157894), (11, 0.8612371838687629), (12, 0.8459193438140807), (13, 0.8462611073137389), (14, 0.8315926179084074), (15, 0.8629938482570062)]}
[(0, 85.452865064695), (1

In [8]:
split = [0.6, 0.05, 0.05, 0.05, 0.05, 0.04, 0.04, 0.04, 0.04, 0.04]
do_fl(split)

Fold 1:


WARNING flwr 2023-09-04 18:43:04,636 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 18:43:04,638 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


torch.Size([3300, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([240, 22])
torch.Size([200, 22])
Data partitioned across 10 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 10 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.
(launch_and_fit pid=30360) Client 5: training for 4 iterations/updates [repeated 4x across cluster]


(launch_and_fit pid=30360) Windows fatal exception: code 0xc0000139 [repeated 6x across cluster]
(launch_and_fit pid=30360)  [repeated 7x across cluster]
(launch_and_fit pid=30360)   File "<frozen importlib._bootstrap>", line 241 in _call_with_frames_removed [repeated 54x across cluster]
(launch_and_fit pid=30360)   File "<frozen importlib._bootstrap_external>", line 883 in exec_module [repeated 42x across cluster]
(launch_and_fit pid=30360)   File "<frozen importlib._bootstrap>", line 688 in _load_unlocked [repeated 42x across cluster]
(launch_and_fit pid=30360)   File "<frozen importlib._bootstrap>", line 1006 in _find_and_load_unlocked [repeated 47x across cluster]
(launch_and_fit pid=30360)   File "<frozen importlib._bootstrap>", line 1027 in _find_and_load [repeated 47x across cluster]
(launch_and_fit pid=30360)   File "<frozen importlib._bootstrap>", line 1078 in _handle_fromlist [repeated 6x across cluster]
(launch_and_fit pid=30360) Stack (most recent call first): [repeated 2x 

(launch_and_get_parameters pid=28668) multiclass client


...

Evaluation on the server: test_mse=4.8965, test_auc=0.8434
Configuring round 15
(launch_and_fit pid=28668) Client 9: training for 4 iterations/updates [repeated 4x across cluster]
(launch_and_fit pid=28908) multiclass client
(launch_and_fit pid=9588) multiclass client
(launch_and_fit pid=28908) Client 6 aggregated_trees length 5
(launch_and_fit pid=28908) Client 6: recieved 5 trees
(launch_and_fit pid=28668) multiclass client [repeated 3x across cluster]
(launch_and_fit pid=28948) Client 0 aggregated_trees length 5 [repeated 4x across cluster]
(launch_and_fit pid=28948) Client 0: recieved 5 trees [repeated 4x across cluster]
(launch_and_fit pid=14464) Client 4: training for 4 iterations/updates
(launch_and_fit pid=28948) Client 0: training for 4 iterations/updates [repeated 4x across cluster]


DEBUG flwr 2023-09-04 20:01:41,487 | fl_server.py:241 | fit_round 15 received 5 results and 0 failures


Server side aggregated 5 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 20:02:20,723 | fl_server.py:112 | fit progress: (15, 5.622920517560074, {'auc': 0.8023513328776486}, 4683.650847199984)
INFO flwr 2023-09-04 20:02:20,726 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 20:02:20,727 | fl_server.py:140 | FL finished in 4683.654990899988
INFO flwr 2023-09-04 20:02:20,729 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 20:02:20,731 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 20:02:20,733 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 20:02:20,735 | app.py:228 | app_fit: losses_centralized [(0, 124.14417744916821), (1, 5.8317929759704255), (2, 3.1700554528650646), (3, 2.521256931608133), (4, 6.534195933456562), (5, 7.720887245841035), (6, 5.208872458410351), (7, 4.519408502772643), (8, 6.77079482439926), (9, 5.56007393715342), (10, 5.153419593345657), (11, 5.9500924214417745), (12, 5.44731977818854), (13, 6.384473197781886), (14,

Evaluation on the server: test_mse=5.6229, test_auc=0.8024
History (loss, centralized):
	round 0: 124.14417744916821
	round 1: 5.8317929759704255
	round 2: 3.1700554528650646
	round 3: 2.521256931608133
	round 4: 6.534195933456562
	round 5: 7.720887245841035
	round 6: 5.208872458410351
	round 7: 4.519408502772643
	round 8: 6.77079482439926
	round 9: 5.56007393715342
	round 10: 5.153419593345657
	round 11: 5.9500924214417745
	round 12: 5.44731977818854
	round 13: 6.384473197781886
	round 14: 4.896487985212569
	round 15: 5.622920517560074
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.8090020505809978), (2, 0.8815447710184552), (3, 0.8873684210526315), (4, 0.812734107997266), (5, 0.7596377306903622), (6, 0.8457347915242652), (7, 0.801948051948052), (8, 0.8106356801093643), (9, 0.8577853725222148), (10, 0.7876213260423787), (11, 0.8169241285030758), (12, 0.8131647300068353), (13, 0.8421941216678059), (14, 0.8434244702665756), (15, 0.8023513328776486)]}
[(0, 124.14417744916821), 

In [9]:
split = [0.8, 0.03, 0.03, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02]
do_fl(split)

Fold 1:


WARNING flwr 2023-09-04 20:02:20,987 | app.py:210 | Both server and strategy were provided, ignoring strategy
INFO flwr 2023-09-04 20:02:20,989 | app.py:145 | Starting Flower simulation, config: ServerConfig(num_rounds=15, round_timeout=None)


torch.Size([4500, 22])
torch.Size([120, 22])
torch.Size([120, 22])
torch.Size([120, 22])
torch.Size([120, 22])
torch.Size([120, 22])
torch.Size([120, 22])
torch.Size([100, 22])
torch.Size([100, 22])
torch.Size([100, 22])
Data partitioned across 10 clients and 0.0 of local dataset reserved for validation.
FL experiment configured for 15 rounds with 10 client in the pool.
FL round will proceed with 50.0% of clients sampled, at least 2.


(launch_and_fit pid=14464) Windows fatal exception: code 0xc0000139 [repeated 6x across cluster]
(launch_and_fit pid=14464)  [repeated 6x across cluster]
(launch_and_fit pid=14464)   File "<frozen importlib._bootstrap>", line 241 in _call_with_frames_removed [repeated 108x across cluster]
(launch_and_fit pid=14464)   File "<frozen importlib._bootstrap_external>", line 883 in exec_module [repeated 84x across cluster]
(launch_and_fit pid=14464)   File "<frozen importlib._bootstrap>", line 688 in _load_unlocked [repeated 84x across cluster]
(launch_and_fit pid=14464)   File "<frozen importlib._bootstrap>", line 1006 in _find_and_load_unlocked [repeated 96x across cluster]
(launch_and_fit pid=14464)   File "<frozen importlib._bootstrap>", line 1027 in _find_and_load [repeated 95x across cluster]
(launch_and_fit pid=14464)   File "<frozen importlib._bootstrap>", line 1078 in _handle_fromlist [repeated 12x across cluster]
(launch_and_fit pid=14464) Stack (most recent call first): [repeated 4

(launch_and_get_parameters pid=20932) multiclass client
serverside task_type multiclass
test  multiclass


...

Evaluation on the server: test_mse=7.1553, test_auc=0.7614
Configuring round 15
(launch_and_fit pid=20932) multiclass client
(launch_and_fit pid=20932) Client 7 aggregated_trees length 5
(launch_and_fit pid=20932) Client 7: recieved 5 trees
(launch_and_fit pid=6956) multiclass client [repeated 4x across cluster]
(launch_and_fit pid=30688) Client 0 aggregated_trees length 5 [repeated 4x across cluster]
(launch_and_fit pid=30688) Client 0: recieved 5 trees [repeated 4x across cluster]
(launch_and_fit pid=15424) Client 3: training for 4 iterations/updates
(launch_and_fit pid=30688) Client 0: training for 4 iterations/updates [repeated 4x across cluster]


DEBUG flwr 2023-09-04 20:52:18,033 | fl_server.py:241 | fit_round 15 received 5 results and 0 failures


Server side aggregated 5 trees.
serverside task_type multiclass
test  multiclass


INFO flwr 2023-09-04 20:53:45,672 | fl_server.py:112 | fit progress: (15, 4.11090573012939, {'auc': 0.8841626794258373}, 3034.200123799994)
INFO flwr 2023-09-04 20:53:45,674 | fl_server.py:159 | evaluate_round 15: no clients selected, cancel
INFO flwr 2023-09-04 20:53:45,675 | fl_server.py:140 | FL finished in 3034.2030265000067
INFO flwr 2023-09-04 20:53:45,676 | app.py:225 | app_fit: losses_distributed []
INFO flwr 2023-09-04 20:53:45,678 | app.py:226 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-09-04 20:53:45,679 | app.py:227 | app_fit: metrics_distributed {}
INFO flwr 2023-09-04 20:53:45,680 | app.py:228 | app_fit: losses_centralized [(0, 54.76155268022181), (1, 8.900184842883549), (2, 12.033271719038817), (3, 7.382624768946395), (4, 7.77818853974122), (5, 7.160813308687615), (6, 8.018484288354898), (7, 7.835489833641405), (8, 6.536044362292052), (9, 8.43807763401109), (10, 9.068391866913124), (11, 5.88724584103512), (12, 3.9038817005545288), (13, 6.238447319778189), (14, 7

Evaluation on the server: test_mse=4.1109, test_auc=0.8842
History (loss, centralized):
	round 0: 54.76155268022181
	round 1: 8.900184842883549
	round 2: 12.033271719038817
	round 3: 7.382624768946395
	round 4: 7.77818853974122
	round 5: 7.160813308687615
	round 6: 8.018484288354898
	round 7: 7.835489833641405
	round 8: 6.536044362292052
	round 9: 8.43807763401109
	round 10: 9.068391866913124
	round 11: 5.88724584103512
	round 12: 3.9038817005545288
	round 13: 6.238447319778189
	round 14: 7.155268022181146
	round 15: 4.11090573012939
History (metrics, centralized):
{'auc': [(0, 0.5), (1, 0.6473820915926178), (2, 0.7590840738209159), (3, 0.7698974709501027), (4, 0.7715652768284349), (5, 0.7758509911141491), (6, 0.7881408065618591), (7, 0.8207450444292551), (8, 0.8429938482570062), (9, 0.7012781954887217), (10, 0.7560902255639098), (11, 0.7850922761449076), (12, 0.8580314422419686), (13, 0.7918728639781271), (14, 0.7613943950786055), (15, 0.8841626794258373)]}
[(0, 54.76155268022181), (1